In [9]:
# === Celda -Limpieza: Limpieza segura del entorno antes del setup ===
import os, shutil, glob, sys, subprocess

print("🧹 Iniciando limpieza del entorno de Colab...\n")

# 1️⃣ Eliminar cachés comunes (Hugging Face, Torch, etc.)
for cache_dir in [
    "/root/.cache/huggingface",
    "/root/.cache/torch/sentence_transformers",
    "/root/.cache/torch/transformers",
    "/content/hf_cache"
]:
    if os.path.exists(cache_dir):
        print(" - Borrando caché:", cache_dir)
        shutil.rmtree(cache_dir, ignore_errors=True)

# 2️⃣ Desinstalar posibles restos conflictivos
subprocess.run([
    sys.executable, "-m", "pip", "uninstall", "-y",
    "transformers", "tokenizers", "huggingface-hub", "sentence-transformers"
], check=False)

# 3️⃣ Borrar distribuciones dañadas (casos como "~cipy")
for p in glob.glob("/usr/local/lib/python3.12/dist-packages/~cipy*"):
    print(" - Borrando resto inválido:", p)
    shutil.rmtree(p, ignore_errors=True)

# 4️⃣ Mostrar versiones base del entorno
print("\n🔍 Versión de Python:", sys.version)
print("📦 Paquetes relevantes actualmente instalados:\n")
subprocess.run([sys.executable, "-m", "pip", "list"], check=False)

print("\n✅ Limpieza completada. Ahora ejecuta la Celda 0 (diagnóstico).")

🧹 Iniciando limpieza del entorno de Colab...

 - Borrando caché: /root/.cache/huggingface

🔍 Versión de Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
📦 Paquetes relevantes actualmente instalados:


✅ Limpieza completada. Ahora ejecuta la Celda 0 (diagnóstico).


In [10]:
# ============================================================
# Celda 0: Diagnóstico completo del entorno (LangChain + HF)
# ============================================================

import importlib, pkgutil, sys

def ver(mod):
    """Imprime versión del módulo si está instalado."""
    try:
        m = importlib.import_module(mod)
        print(f"{mod:28s}", getattr(m, "__version__", "(sin __version__)"))
    except Exception as e:
        print(f"{mod:28s}", "— no instalado —", "|", e)

print("Python:", sys.version.split()[0])
print("\n📦 Versiones detectadas:")
for mod in [
    "numpy", "scipy", "sklearn", "torch",
    "transformers", "tokenizers",
    "huggingface_hub", "sentence_transformers",
    "langchain", "langchain_community", "langchain_huggingface"
]:
    ver(mod)

# ============================================================
# Verificación de que transformers está completamente instalado
# ============================================================
print("\n🔍 Verificando instalación completa de transformers...")
try:
    from transformers.models.bert.configuration_bert import BertConfig
    print("✅ Módulo BertConfig importado correctamente")
except ImportError as e:
    print(f"⚠️ Error al importar BertConfig: {e}")
    print("   Esto indica que transformers no está completamente instalado.")
    print("   Ejecuta la Celda 1 (instalación) nuevamente o reinicia el runtime.")

# ============================================================
# Comprobación automática de HuggingFaceEmbeddings disponible
# ============================================================
print("\n🔍 Comprobando integración de LangChain + HuggingFace...")

try:
    from langchain_huggingface import HuggingFaceEmbeddings
    origen = "langchain_huggingface (moderno ✅)"
    HuggingFaceEmbeddings_available = True
except ModuleNotFoundError:
    try:
        from langchain_community.embeddings import HuggingFaceEmbeddings
        origen = "langchain_community.embeddings (clásico ⚙️)"
        HuggingFaceEmbeddings_available = True
    except ModuleNotFoundError:
        HuggingFaceEmbeddings = None
        HuggingFaceEmbeddings_available = False
        origen = "❌ Ningún módulo de HuggingFaceEmbeddings disponible"

print("Origen del wrapper:", origen)

# ============================================================
# Prueba funcional (si existe HuggingFaceEmbeddings)
# ============================================================
if HuggingFaceEmbeddings_available and HuggingFaceEmbeddings is not None:
    try:
        import torch
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print("Dispositivo:", device)

        embeddings_model = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={"device": device},
            encode_kwargs={"normalize_embeddings": True},
        )

        test_text = "La inteligencia artificial aprende patrones del lenguaje humano."
        vec = embeddings_model.embed_query(test_text)
        print("\n✅ Embeddings funcionando correctamente")
        print("Dimensión del embedding:", len(vec))
    except Exception as e:
        print(f"\n❌ Error al probar embeddings: {e}")
        print("   Posible causa: transformers no está completamente instalado.")
        print("   Solución: Reinicia el runtime y ejecuta las celdas en orden.")
else:
    print("\n⚠️ No se pudo inicializar HuggingFaceEmbeddings.")
    print("Ejecuta la Celda 1 (instalación) para instalar los paquetes necesarios.")


Python: 3.12.12

📦 Versiones detectadas:
numpy                        2.0.2
scipy                        1.16.3
sklearn                      1.6.1
torch                        2.8.0+cu126
transformers                 4.57.1
tokenizers                   0.22.1
huggingface_hub              0.36.0
sentence_transformers        5.1.2
langchain                    0.3.27
langchain_community          0.4.1
langchain_huggingface        (sin __version__)

🔍 Verificando instalación completa de transformers...
✅ Módulo BertConfig importado correctamente

🔍 Comprobando integración de LangChain + HuggingFace...
Origen del wrapper: langchain_huggingface (moderno ✅)
Dispositivo: cpu


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


✅ Embeddings funcionando correctamente
Dimensión del embedding: 384


In [11]:
# CELDA 1: Instalación de dependencias
# ============================================================
# IMPORTANTE: Si ves warnings sobre dependencias, son normales en Colab.
# Si hay errores críticos, reinicia el runtime (Runtime > Restart runtime)
# ============================================================

# 1) Fijar versiones de dependencias base para minimizar conflictos con Colab
# Estas versiones son compatibles con los paquetes preinstalados en Colab
print("📦 Configurando versiones compatibles de dependencias base...")
%pip install --quiet \
  "requests==2.32.4" \
  "fsspec==2025.3.0"

# 2) Reinstalar transformers completamente para evitar problemas de módulos faltantes
# El error "No module named 'transformers.models.bert.configuration_bert'" indica
# que transformers no está completamente instalado
print("🔄 Reinstalando transformers para corregir módulos faltantes...")
%pip install --quiet --force-reinstall --no-cache-dir \
  "transformers>=4.45.0,<5.0.0" \
  "tokenizers>=0.20.0"

# 3) Instalar sentence-transformers y huggingface-hub
%pip install --quiet --upgrade \
  "sentence-transformers>=2.7.0" \
  "huggingface-hub>=0.36.0"

# 4) Instalar LangChain y conectores (versión 1.x para compatibilidad)
%pip install --quiet --upgrade \
  "langchain>=1.0.0" \
  "langchain-huggingface>=0.0.3" \
  "langchain-community>=0.2.15" \
  "langchain-openai>=0.1.0"

# 5) Re-fijar requests y fsspec al final para asegurar que no se actualicen
# (algunos paquetes pueden intentar actualizarlos durante la instalación)
%pip install --quiet --no-deps "requests==2.32.4" "fsspec==2025.3.0"

# 3) Verificación rápida
import importlib.util
import sys

print("=" * 60)
print("📦 VERIFICACIÓN DE INSTALACIÓN")
print("=" * 60)

try:
    import transformers
    print(f"✅ transformers: {transformers.__version__}")
except Exception as e:
    print(f"❌ transformers: {e}")

try:
    import tokenizers
    print(f"✅ tokenizers: {tokenizers.__version__}")
except Exception as e:
    print(f"❌ tokenizers: {e}")

try:
    import sentence_transformers
    print(f"✅ sentence-transformers: {sentence_transformers.__version__}")
except Exception as e:
    print(f"❌ sentence-transformers: {e}")

try:
    import huggingface_hub
    print(f"✅ huggingface-hub: {huggingface_hub.__version__}")
except Exception as e:
    print(f"❌ huggingface-hub: {e}")

try:
    import langchain
    print(f"✅ langchain: {langchain.__version__}")
except Exception as e:
    print(f"❌ langchain: {e}")

try:
    import langchain_huggingface
    print(f"✅ langchain-huggingface: disponible")
except Exception as e:
    print(f"❌ langchain-huggingface: {e}")

try:
    import langchain_openai
    print(f"✅ langchain-openai: disponible")
except Exception as e:
    print(f"❌ langchain-openai: {e}")

# Verificar módulo BERT
bert_available = importlib.util.find_spec("transformers.models.bert.modeling_bert") is not None
print(f"✅ BERT module: {'disponible' if bert_available else 'no disponible'}")

print("=" * 60)
print("✅ Instalación completada")
print("\n📝 NOTA SOBRE WARNINGS:")
print("   Los warnings sobre dependencias (numpy, fsspec, etc.) son NORMALES en Colab.")
print("   Son informativos y generalmente NO afectan el funcionamiento.")
print("   Solo preocúpate si ves ERRORES reales al ejecutar el código.")
print("\n   Si hay errores, reinicia el runtime: Runtime > Restart runtime")
print("=" * 60)


📦 Configurando versiones compatibles de dependencias base...
🔄 Reinstalando transformers para corregir módulos faltantes...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 173.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 209.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 243.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 148.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 216.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 219.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 162.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 153.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 232.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 222.4 MB/s eta

In [12]:
# Celda 2: confirmar que todo quedó consistente
import importlib, importlib.util

def ver(mod):
    m = importlib.import_module(mod)
    print(f"{mod:22s}", getattr(m, "__version__", "(sin __version__)"))

for mod in ["transformers","tokenizers","huggingface_hub","sentence_transformers"]:
    ver(mod)

# Chequeo de BERT presente
import transformers, importlib.util
print("BERT presente ->", importlib.util.find_spec("transformers.models.bert") is not None)


transformers           4.57.1
tokenizers             0.22.1
huggingface_hub        0.36.0
sentence_transformers  5.1.2
BERT presente -> True


In [13]:
# Celda 3: prueba de humo; si falla, limpia caché y reintenta una vez
from sentence_transformers import SentenceTransformer
import os, shutil

def try_load(clean_cache=False):
    cache_dir = "/content/hf_cache" if clean_cache else None
    if clean_cache:
        for p in ["/content/hf_cache", os.path.expanduser("~/.cache/huggingface")]:
            if os.path.exists(p):
                print("Limpiando caché:", p); shutil.rmtree(p, ignore_errors=True)
    m = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2",
                            cache_folder=cache_dir, trust_remote_code=False)
    v = m.encode(["hola","mundo"], normalize_embeddings=True)
    print("OK; dimensión:", len(v[0]))

try:
    try_load(clean_cache=False)
except Exception as e:
    print("⚠️ Falla inicial:", e)
    print("→ Reintentando con caché limpia…")
    try_load(clean_cache=True)


OK; dimensión: 384


In [14]:
###Prueba de que las librerias quedaron bien instaladas y funcionan respecto a langchain

from langchain_huggingface import HuggingFaceEmbeddings
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

# Versión sin cache_dir (usa la ruta por defecto)
embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": device},
    encode_kwargs={"normalize_embeddings": True}
)

# Prueba simple
test_text = "La inteligencia artificial aprende patrones del lenguaje humano."
vec = embeddings_model.embed_query(test_text)
print("✅ LangChain conectado con éxito.")
print("Dimensión del embedding:", len(vec))


Usando dispositivo: cpu
✅ LangChain conectado con éxito.
Dimensión del embedding: 384


In [15]:
# @title
# ============================================
# 1) Montar Google Drive
# ============================================
from google.colab import drive

drive.mount('/content/drive')
print("✅ Drive montado.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive montado.


In [16]:
# @title
# ============================================
# 2) Definir las rutas base en Drive
#    Ajustadas a tu estructura:
#    Mi unidad / Colab Notebooks / Tarea3-IA / ...
# ============================================
import os

# Ruta base a "Mi unidad"
BASE_DRIVE = "/content/drive/MyDrive"

# Carpeta raíz del proyecto de la tarea
PROYECTO_DIR = os.path.join(BASE_DRIVE, "Colab Notebooks", "Tarea3-IA")

# Carpetas específicas
METADATA_FILE = os.path.join(PROYECTO_DIR, "MetadataRAW.csv")
PDFS_DIR = os.path.join(PROYECTO_DIR, "RepositorioApuntesPdf")

print("📁 Proyecto:", PROYECTO_DIR)
print("📁 Metadata:", METADATA_FILE)
print("📁 PDFs:", PDFS_DIR)


📁 Proyecto: /content/drive/MyDrive/Colab Notebooks/Tarea3-IA
📁 Metadata: /content/drive/MyDrive/Colab Notebooks/Tarea3-IA/MetadataRAW.csv
📁 PDFs: /content/drive/MyDrive/Colab Notebooks/Tarea3-IA/RepositorioApuntesPdf


In [17]:
# @title
# ============================================
# 3) Verificar que las carpetas existen
#    (esto ayuda a detectar typos en el nombre)
# ============================================

def check_dir(path, name):
    if os.path.exists(path):
        print(f"✅ {name} encontrada: {path}")
    else:
        print(f"❌ {name} NO encontrada. Revisa el nombre en Drive: {path}")

check_dir(PROYECTO_DIR, "Carpeta del proyecto")
check_dir(METADATA_FILE, "Archivo de metadata")
check_dir(PDFS_DIR, "Carpeta de PDFs")


✅ Carpeta del proyecto encontrada: /content/drive/MyDrive/Colab Notebooks/Tarea3-IA
✅ Archivo de metadata encontrada: /content/drive/MyDrive/Colab Notebooks/Tarea3-IA/MetadataRAW.csv
✅ Carpeta de PDFs encontrada: /content/drive/MyDrive/Colab Notebooks/Tarea3-IA/RepositorioApuntesPdf


In [18]:
# @title
# ============================================
# 4) Listar los PDFs disponibles
#    (esto confirma que Colab está viendo tu carpeta)
# ============================================

pdf_files = [f for f in os.listdir(PDFS_DIR) if f.lower().endswith(".pdf")]
print(f"📚 PDFs encontrados: {len(pdf_files)}")
for f in pdf_files:
    print("  -", f)


📚 PDFs encontrados: 46
  - 5_Semana_AI_20250904_2.pdf
  - 6_Semana_AI_20250911_2.pdf
  - 11_Semana_AI_20251016_4.pdf
  - 11_Semana_AI_20251014_3.pdf
  - 10_SEMANA_AI_20251007_1-222887296.pdf
  - 7_Semana_AI_20250916_2.pdf
  - 12_SEMANA_AI_20251021_3.pdf
  - 2_Semana_AI_20250812_3.pdf
  - 11_Semana_AI_20251014_2.pdf
  - 8_Semana_AI_20250925_2.pdf
  - 11_Semana_AI_20251014_1.pdf
  - 5_Semana_AI_20250904_1.pdf
  - 10_SEMANA_AI_20251009_1.pdf
  - 12_SEMANA_AI_20251021_4.pdf
  - 8_Semana_AI_20250923_1.pdf
  - 1_Semana_AI_20250807_2.pdf
  - 2_Semana_AI_20250814_1.pdf
  - 2_Semana_AI_20250814_2.pdf
  - 5_Semana_AI_20250902_2.pdf
  - 6_Semana_AI_20250911_1.pdf
  - 5_Semana_AI_20250902_1.pdf
  - 11_SEMANA_AI_20251016_2.pdf
  - 4_SEMANA_AI_20250826_1.pdf
  - 6_Semana_AI_20250909_2-220676337.pdf
  - 8_Semana_AI_20250925_1.pdf
  - 7_Semana_AI_20250918_1.pdf
  - 12_SEMANA_AI_20251021_1.pdf
  - 3_Semana_AI_20250819_2.pdf
  - 4_Semana_AI_20250828_1.pdf
  - 7_Semana_AI_20250918_2.pdf
  - 4_SEMANA_AI_2

In [19]:
# @title
# ============================================
# 5) Cargar archivo de metadata
#    Columnas esperadas: id_doc, nombre_archivo, autor, fecha, tema
# ============================================

import os
import pandas as pd

if os.path.exists(METADATA_FILE):
    # Leer tal cual, como texto (sin parsear fechas)
    df_meta = pd.read_csv(METADATA_FILE, dtype=str, keep_default_na=False)
    print(f"✅ Metadata CSV cargada correctamente ({len(df_meta)} filas) desde:\n{METADATA_FILE}\n")

    # Chequeo suave de columnas (sin modificar nada)
    expected = ["id_doc", "nombre_archivo", "autor", "fecha", "tema"]
    missing = [c for c in expected if c not in df_meta.columns]
    if missing:
        print("⚠️ Faltan columnas esperadas:", missing)
    else:
        print("📑 Columnas detectadas:", list(df_meta.columns))

        df_meta["fecha"] = pd.to_datetime(df_meta["fecha"], errors="coerce")


    # Preview
    display(df_meta.head(10))
else:
    print("⚠️ No se encontró el archivo 'metadata.csv' en la carpeta Metadata:", METADATA_FILE)


✅ Metadata CSV cargada correctamente (46 filas) desde:
/content/drive/MyDrive/Colab Notebooks/Tarea3-IA/MetadataRAW.csv

📑 Columnas detectadas: ['id_doc', 'nombre_archivo', 'autor', 'fecha', 'tema']


,id_doc,nombre_archivo,autor,fecha,tema
0,DOC_001,1_SEMANA_AI_20250807_1.pdf,Rodolfo David Acuña López,2025-08-07,Principios fundamentales de la inteligencia ar...
1,DOC_002,1_Semana_AI_20250807_2.pdf,Fernando Daniel Brenes Reyes,2025-08-07,Aplicaciones de la inteligencia artificial y m...
2,DOC_003,2_SEMANA_AI_20250812_1.pdf,Priscilla Jiménez Salgado,2025-08-12,Introducción a machine learning y deep learnin...
3,DOC_004,2_Semana_AI_20250812_3.pdf,Luis Alfredo González Sánchez,2025-08-12,Resumen de conceptos clave de IA y enfoques de...
4,DOC_005,2_Semana_AI_20250814_1.pdf,Kendall Rodríguez Camacho,2025-08-14,Introducción a álgebra lineal aplicada con Pyt...
5,DOC_006,2_Semana_AI_20250814_2.pdf,Jose Pablo Quesada Rodríguez,2025-08-14,"Resumen detallado sobre tipos de aprendizaje, ..."
6,DOC_007,3_Semana_AI_20250819_1.pdf,Javier Rojas Rojas,2025-08-19,Revisión de álgebra lineal y aprendizaje super...
7,DOC_008,3_Semana_AI_20250819_2.pdf,Mariana Quesada Sánchez,2025-08-19,Repaso de álgebra lineal y fundamentos del apr...
8,DOC_009,3_Semana_AI_20250821_1.pdf,Julio Varela Venegas,2025-08-21,Aplicación del álgebra lineal y la programació...
9,DOC_010,4_SEMANA_AI_20250826_1.pdf,Andrés Sánchez Rojas,2025-08-26,Implementación del algoritmo KNN y fundamentos...


In [20]:
# @title
# ============================================================
# COMPAÑERO 1 – DATOS Y PREPROCESAMIENTO (PASO 2 COMPLETO)
# Extraer texto, normalizar y segmentar (A: chunks fijos, B: encabezados)
# Requiere:
#  - Variables definidas antes: PROYECTO_DIR, METADATA_FILE, PDFS_DIR
#  - METADATA_FILE con columnas: id_doc, nombre_archivo, autor, fecha, tema
# Salidas (para Compañero 2):
#  - dataset/base_documentos.jsonl / .parquet
#  - dataset/seg_a.jsonl / dataset/seg_b.jsonl
#  - dataset/txt_por_doc/DOC_###.txt (opcional)
#  - dataset/preprocesamiento_decisiones.md
# ============================================================

!pip install --quiet pdfplumber pandas pyarrow

import os, re, json, unicodedata
from pathlib import Path
import pdfplumber
import pandas as pd

# ---------- CONFIGURACIÓN ----------
OUT_DIR = os.path.join(PROYECTO_DIR, "dataset")
TXT_DIR = os.path.join(OUT_DIR, "txt_por_doc")
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)
Path(TXT_DIR).mkdir(parents=True, exist_ok=True)

BASE_DOCS_JSONL   = os.path.join(OUT_DIR, "base_documentos.jsonl")
BASE_DOCS_PARQUET = os.path.join(OUT_DIR, "base_documentos.parquet")
SEG_A_JSONL       = os.path.join(OUT_DIR, "seg_a.jsonl")
SEG_B_JSONL       = os.path.join(OUT_DIR, "seg_b.jsonl")
NOTAS_PREPROC     = os.path.join(OUT_DIR, "preprocesamiento_decisiones.md")

# (Ajusta si querés otros tamaños)
CHUNK_WORDS   = 400    # tamaño del chunk (en palabras aproximado)
CHUNK_OVERLAP = 80     # solapamiento entre chunks

# Si existen salidas previas, limpirlas (evita duplicados al re-ejecutar)
for p in [BASE_DOCS_JSONL, BASE_DOCS_PARQUET, SEG_A_JSONL, SEG_B_JSONL, NOTAS_PREPROC]:
    try:
        if os.path.exists(p): os.remove(p)
    except Exception:
        pass

# ---------- NORMALIZACIÓN ----------
def normalize_unicode_nfc(text: str) -> str:
    return unicodedata.normalize("NFC", text)

def strip_control_chars(text: str) -> str:
    # Deja \n y \t; elimina otros de control
    return "".join(ch for ch in text if ch in ("\n","\t") or ord(ch) >= 32)

def standardize_quotes_dashes(text: str) -> str:
    repl = {
        "“": "\"", "”": "\"", "„": "\"", "«": "\"", "»": "\"",
        "’": "'", "´": "'", "‘": "'",
        "‐": "-", "–": "-", "—": "-", "−": "-",
        "…": "...", "•": "-", "·": "-"
    }
    for a, b in repl.items():
        text = text.replace(a, b)
    return text

def remove_hyphen_linebreaks(text: str) -> str:
    # Une palabras cortadas por guion al final de línea: "infor-\nmación" -> "información"
    return re.sub(r"(\w+)-\n(\w+)", r"\1\2", text)

def collapse_whitespace(text: str) -> str:
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

def to_lower(text: str) -> str:
    return text.lower()

def normalize_text_pipeline(raw: str) -> str:
    if not raw: return ""
    t = normalize_unicode_nfc(raw)
    t = strip_control_chars(t)
    t = standardize_quotes_dashes(t)
    t = remove_hyphen_linebreaks(t)
    t = collapse_whitespace(t)
    t = to_lower(t)
    return t

# ---------- SEGMENTACIÓN ----------
def segment_fixed_overlap(text: str, words_per_chunk=CHUNK_WORDS, overlap=CHUNK_OVERLAP):
    """Segmentación A: chunks por palabras con solapamiento."""
    words = text.split()
    if not words: return []
    chunks = []
    i = 0
    idx = 0
    while i < len(words):
        chunk_words = words[i:i+words_per_chunk]
        chunk_text = " ".join(chunk_words).strip()
        if chunk_text:
            chunks.append((idx, chunk_text))
            idx += 1
        i += max(1, words_per_chunk - overlap)
    return chunks

_HEADING_RE = re.compile(
    r"""^(
        abstract\b|
        resumen\b|
        introducci[oó]n\b|
        conclusi[oó]n\b|
        referencias\b|
        agradecimientos\b|
        related\ work\b|
        i{1,3}\.|iv\.|v\.|vi\.|vii\.|viii\.|ix\.|x\.|      # romanos
        \d+\.\s|                                          # 1. 2. 3.
        [A-Z][A-Z0-9\s\-\&]{3,}$                          # línea MAYÚSCULAS (título)
    )""",
    re.IGNORECASE | re.VERBOSE
)

def segment_by_headings(text: str, min_section_words=120):
    """Segmentación B: por encabezados; fusiona secciones pequeñas."""
    lines = [l.strip() for l in text.splitlines()]
    # Marcar índices de líneas que parecen encabezado
    header_idx = [i for i, line in enumerate(lines) if line and _HEADING_RE.match(line)]
    # Siempre incluir inicio y fin
    if 0 not in header_idx: header_idx = [0] + header_idx
    if len(lines) - 1 not in header_idx: header_idx = header_idx + [len(lines) - 1]
    header_idx = sorted(set(header_idx))

    # Cortar por rangos
    raw_sections = []
    for a, b in zip(header_idx, header_idx[1:]):
        sec = "\n".join(lines[a:b]).strip()
        if sec: raw_sections.append(sec)
    # Añadir último trozo
    tail = "\n".join(lines[header_idx[-1]:]).strip()
    if tail: raw_sections.append(tail)

    # Fusionar secciones demasiado pequeñas
    merged = []
    buff = []
    wcount = 0
    for sec in raw_sections:
        wc = len(sec.split())
        if wcount + wc < min_section_words:
            buff.append(sec); wcount += wc
        else:
            if buff:
                buff.append(sec)
                merged.append("\n\n".join(buff).strip())
                buff, wcount = [], 0
            else:
                merged.append(sec)
                buff, wcount = [], 0
    if buff:
        merged.append("\n\n".join(buff).strip())

    # Indexar
    return [(i, s) for i, s in enumerate(merged)]

# ---------- CARGA METADATA ----------
assert os.path.exists(METADATA_FILE), f"No existe METADATA_FILE: {METADATA_FILE}"
df_meta = pd.read_csv(METADATA_FILE, dtype=str, keep_default_na=False)
for col in ["id_doc","nombre_archivo","autor","fecha","tema"]:
    if col not in df_meta.columns:
        raise ValueError(f"Falta columna en metadata: {col}")

# índice rápido por nombre de archivo (case-insensitive)
meta_idx = {str(n).strip().lower(): i for i, n in enumerate(df_meta["nombre_archivo"])}

# ---------- RECORRIDO PDFs ----------
base_registros = []
seg_a_registros = []
seg_b_registros = []
errores = []

pdf_files_sorted = sorted([f for f in os.listdir(PDFS_DIR) if f.lower().endswith(".pdf")])

for fname in pdf_files_sorted:
    key = fname.strip().lower()
    if key not in meta_idx:
        errores.append((fname, "no_encontrado_en_metadata"))
        print(f"⚠️  {fname}: no aparece en 'nombre_archivo' de la metadata; se omite.")
        continue

    row   = df_meta.iloc[meta_idx[key]]
    iddoc = row["id_doc"]; autor=row["autor"]; fecha=row["fecha"]; tema=row["tema"]
    pdf_path = os.path.join(PDFS_DIR, fname)

    # Extraer texto del PDF
    try:
        pages = []
        with pdfplumber.open(pdf_path) as pdf:
            for p in pdf.pages:
                pages.append(p.extract_text() or "")
        full_text = "\n".join(pages)
    except Exception as e:
        errores.append((fname, f"error_lectura_pdf:{e}"))
        print(f"❌ Error leyendo {fname}: {e}")
        continue

    # Normalizar
    texto_limpio = normalize_text_pipeline(full_text)

    # Guardar TXT por doc (útil para inspección)
    with open(os.path.join(TXT_DIR, f"{iddoc}.txt"), "w", encoding="utf-8") as f:
        f.write(texto_limpio)

    # Registro base (documento completo)
    base_registros.append({
        "id_doc": iddoc,
        "nombre_archivo": fname,
        "autor": autor,
        "fecha": str(fecha),
        "tema": tema,
        "texto_original": full_text,
        "texto_limpio": texto_limpio
    })

    # ---------- SEGMENTACIÓN A: chunks fijos ----------
    chunks_a = segment_fixed_overlap(texto_limpio, CHUNK_WORDS, CHUNK_OVERLAP)
    for idx_chunk, chunk_text in chunks_a:
        seg_a_registros.append({
            "id_doc": iddoc,
            "segmentacion": "A",
            "chunk_id": f"{iddoc}_A_{idx_chunk:03d}",
            "idx": idx_chunk,
            "autor": autor,
            "fecha": str(fecha),
            "tema": tema,
            "texto": chunk_text
        })

    # ---------- SEGMENTACIÓN B: encabezados ----------
    sections_b = segment_by_headings(texto_limpio, min_section_words=120)
    for idx_sec, sec_text in sections_b:
        seg_b_registros.append({
            "id_doc": iddoc,
            "segmentacion": "B",
            "chunk_id": f"{iddoc}_B_{idx_sec:03d}",
            "idx": idx_sec,
            "autor": autor,
            "fecha": str(fecha),
            "tema": tema,
            "texto": sec_text
        })

# ---------- GUARDAR SALIDAS ----------
# Base documentos
with open(BASE_DOCS_JSONL, "w", encoding="utf-8") as jf:
    for r in base_registros:
        jf.write(json.dumps(r, ensure_ascii=False) + "\n")

pd.DataFrame(base_registros).to_parquet(BASE_DOCS_PARQUET, index=False)

# Segmentaciones
with open(SEG_A_JSONL, "w", encoding="utf-8") as jf:
    for r in seg_a_registros:
        jf.write(json.dumps(r, ensure_ascii=False) + "\n")

with open(SEG_B_JSONL, "w", encoding="utf-8") as jf:
    for r in seg_b_registros:
        jf.write(json.dumps(r, ensure_ascii=False) + "\n")

# Mini-documentación de decisiones para el informe
notas = f"""# Preprocesamiento y Segmentación (borrador)

**Normalización aplicada**
- Unicode NFC (mantener tildes correctas).
- Limpieza de caracteres de control (excepto \\n y \\t).
- Estandarización de comillas/guiones (“ ” ‘ ’ — – … → " ' - ...).
- Unión de palabras cortadas por guion al fin de línea (e.g., "infor-\\nmación" → "información").
- Colapso de espacios y saltos en blanco excesivos.
- Conversión a minúsculas (para comparar segmentaciones bajo mismas condiciones).

**Segmentación A – Chunks fijos**
- Tamaño ≈ {CHUNK_WORDS} palabras, solapamiento ≈ {CHUNK_OVERLAP}.
- Ventajas: control de longitud, útil para evaluación reproducible.
- Desventajas: puede cortar ideas a mitad.

**Segmentación B – Encabezados/Secciones**
- Reglas: detec. de Abstract/Resumen/Introducción/Conclusión/Referencias, numerales (1., 2., ...), romanos (I., II., ...), títulos en MAYÚSCULAS.
- Se fusionan secciones muy cortas (<120 palabras) con la siguiente para asegurar contexto mínimo.
- Ventajas: mantiene unidades semánticas; Desventajas: depende de patrones editoriales.

**Salidas**
- base_documentos.jsonl / .parquet: texto completo normalizado por documento + metadata (autor/fecha/tema).
- seg_a.jsonl / seg_b.jsonl: fragmentos con `id_doc`, `chunk_id`, `segmentacion`, `idx`, `texto`, y metadata.
- txt_por_doc/: útil para inspección rápida o depurar PDF problemáticos.

**Razonamiento para comparación**
- A: garantiza tamaño estable → resultados de recuperación comparables.
- B: favorece coherencia semántica → potencialmente mejor grounding.
- Compañero 2 debe crear dos índices (A y B) y comparar métricas (recall@k, precisión manual, tiempo respuesta).
"""
with open(NOTAS_PREPROC, "w", encoding="utf-8") as f:
    f.write(notas)

print("✅ Extracción, normalización y segmentación listas")
print(f"🧾 Base (JSONL):  {BASE_DOCS_JSONL}")
print(f"🧱 Base (Parquet): {BASE_DOCS_PARQUET}")
print(f"🔹 Seg A (JSONL):  {SEG_A_JSONL}")
print(f"🔸 Seg B (JSONL):  {SEG_B_JSONL}")
print(f"🗒️  Notas:          {NOTAS_PREPROC}")
if errores:
    print(f"⚠️ Incidencias ({len(errores)}):")
    for e in errores[:12]:
        print("   -", e)
    if len(errores) > 12:
        print("   ...")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 16.1 MB/s eta 0:00:00
✅ Extracción, normalización y segmentación listas
🧾 Base (JSONL):  /content/drive/MyDrive/Colab Notebooks/Tarea3-IA/dataset/base_documentos.jsonl
🧱 Base (Parquet): /content/drive/MyDrive/Colab Notebooks/Tarea3-IA/dataset/base_documentos.parquet
🔹 Seg A (JSONL):  /content/drive/MyDrive/Colab Notebooks/Tarea3-IA/dataset/seg_a.jsonl
🔸 Seg B (JSONL):  /content/drive/MyDrive/Colab Notebooks/Tarea3-IA/dataset/seg_b.jsonl
🗒️  Notas:          /content/drive/MyDrive/Colab Notebooks/Tarea3-IA/dataset/preprocesamiento_decisiones.md


In [21]:
# @title
# ============================================================
# 🔚 CIERRE Priscilla – VERIFICACIÓN FINAL Y RESUMEN
# ============================================================

import os

# Rutas de salida esperadas
OUT_DIR = os.path.join(PROYECTO_DIR, "dataset")
paths = {
    "Metadata": METADATA_FILE,
    "Base documentos JSONL": os.path.join(OUT_DIR, "base_documentos.jsonl"),
    "Base documentos Parquet": os.path.join(OUT_DIR, "base_documentos.parquet"),
    "Segmentación A (chunks fijos)": os.path.join(OUT_DIR, "seg_a.jsonl"),
    "Segmentación B (encabezados)": os.path.join(OUT_DIR, "seg_b.jsonl"),
    "Notas de preprocesamiento": os.path.join(OUT_DIR, "preprocesamiento_decisiones.md"),
}

print("📂 Verificando archivos generados...\n")
for nombre, ruta in paths.items():
    if os.path.exists(ruta):
        print(f"✅ {nombre}: encontrado ({os.path.basename(ruta)})")
    else:
        print(f"⚠️ {nombre}: NO encontrado -> {ruta}")

# Conteo rápido de PDFs y documentos base
pdfs = [f for f in os.listdir(PDFS_DIR) if f.lower().endswith(".pdf")]
pdf_count = len(pdfs)
base_path = paths["Base documentos JSONL"]
base_count = sum(1 for _ in open(base_path, encoding="utf-8")) if os.path.exists(base_path) else 0

print(f"\n📊 PDFs reales: {pdf_count}")
print(f"📄 Documentos procesados: {base_count}")

if pdf_count == base_count:
    print("\n✅ Todos los documentos fueron procesados correctamente.")
else:
    print("\n⚠️ Hay diferencias entre PDFs y registros procesados. Revisa nombres o metadatos.")

print("\n🧭 RESUMEN PARA David:")
print("""
Los datos están listos para generar embeddings:

- dataset/seg_a.jsonl → fragmentos con segmentación A (chunks fijos)
- dataset/seg_b.jsonl → fragmentos con segmentación B (por encabezados)
- MetadataRAW.csv (o metadata.csv) → autor, fecha, tema por documento

Archivos opcionales:
- base_documentos.jsonl/.parquet → textos completos normalizados
- preprocesamiento_decisiones.md → descripción de limpieza y segmentación

David debe usar seg_a.jsonl y seg_b.jsonl
para crear las dos bases vectoriales y comparar su rendimiento.
""")


📂 Verificando archivos generados...

✅ Metadata: encontrado (MetadataRAW.csv)
✅ Base documentos JSONL: encontrado (base_documentos.jsonl)
✅ Base documentos Parquet: encontrado (base_documentos.parquet)
✅ Segmentación A (chunks fijos): encontrado (seg_a.jsonl)
✅ Segmentación B (encabezados): encontrado (seg_b.jsonl)
✅ Notas de preprocesamiento: encontrado (preprocesamiento_decisiones.md)

📊 PDFs reales: 46
📄 Documentos procesados: 46

✅ Todos los documentos fueron procesados correctamente.

🧭 RESUMEN PARA David:

Los datos están listos para generar embeddings:

- dataset/seg_a.jsonl → fragmentos con segmentación A (chunks fijos)
- dataset/seg_b.jsonl → fragmentos con segmentación B (por encabezados)
- MetadataRAW.csv (o metadata.csv) → autor, fecha, tema por documento

Archivos opcionales:
- base_documentos.jsonl/.parquet → textos completos normalizados
- preprocesamiento_decisiones.md → descripción de limpieza y segmentación

David debe usar seg_a.jsonl y seg_b.jsonl
para crear las dos

In [22]:
# ============================================================
# Paso 2: Configuración y carga de datos segmentados (limpio)
# ============================================================

from typing import List, Dict
import os, json

# Rutas de datos (definidas por Compañero 1)
OUT_DIR = os.path.join(PROYECTO_DIR, "dataset")
SEG_A_JSONL = os.path.join(OUT_DIR, "seg_a.jsonl")
SEG_B_JSONL = os.path.join(OUT_DIR, "seg_b.jsonl")

# Verificar que los archivos existen
if not os.path.exists(SEG_A_JSONL):
    raise FileNotFoundError(f"No se encontró {SEG_A_JSONL}. Asegúrate de que Compañero 1 completó su parte.")
if not os.path.exists(SEG_B_JSONL):
    raise FileNotFoundError(f"No se encontró {SEG_B_JSONL}. Asegúrate de que Compañero 1 completó su parte.")

print("Archivos de segmentación encontrados:")
print(f"- Segmentación A: {SEG_A_JSONL}")
print(f"- Segmentación B: {SEG_B_JSONL}")

def load_segmented_data(jsonl_path: str) -> List[Dict]:
    """Carga los datos segmentados desde un archivo JSONL."""
    data = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    return data

# Cargar datos
seg_a_data = load_segmented_data(SEG_A_JSONL)
seg_b_data = load_segmented_data(SEG_B_JSONL)

print("\nDatos cargados:")
print(f"- Segmentación A: {len(seg_a_data)} fragmentos")
print(f"- Segmentación B: {len(seg_b_data)} fragmentos")

# Mostrar ejemplo de un fragmento
if seg_a_data:
    ejemplo = seg_a_data[0]
    print("\nEjemplo de fragmento (Segmentación A):")
    print(f"- chunk_id: {ejemplo.get('chunk_id', 'N/A')}")
    print(f"- id_doc: {ejemplo.get('id_doc', 'N/A')}")
    print(f"- autor: {ejemplo.get('autor', 'N/A')}")
    print(f"- texto (primeros 100 chars): {ejemplo.get('texto', '')[:100]}…")


Archivos de segmentación encontrados:
- Segmentación A: /content/drive/MyDrive/Colab Notebooks/Tarea3-IA/dataset/seg_a.jsonl
- Segmentación B: /content/drive/MyDrive/Colab Notebooks/Tarea3-IA/dataset/seg_b.jsonl

Datos cargados:
- Segmentación A: 227 fragmentos
- Segmentación B: 349 fragmentos

Ejemplo de fragmento (Segmentación A):
- chunk_id: DOC_033_A_000
- id_doc: DOC_033
- autor: María José Chacón Rodríguez
- texto (primeros 100 chars): apuntes ia clase 7/10 gianmarco oporta pe'rez ingenier'ıa en computacio'n instituto tecnolo'gico de …


In [23]:
# ============================================================
# Paso 3: Tokenización y generación de embeddings (estable)
# Modelo: sentence-transformers/all-MiniLM-L6-v2
# ============================================================

import warnings, sys
warnings.filterwarnings("ignore")
import tiktoken

def count_tokens(text: str, model: str = "gpt-3.5-turbo") -> int:
    try:
        enc = tiktoken.encoding_for_model(model)
        return len(enc.encode(text))
    except Exception:
        return max(1, len(text) // 4)

print("Configurando modelo de embeddings...")
print("Modelo objetivo: sentence-transformers/all-MiniLM-L6-v2")
print("Dimensión esperada: 384")

# Detectar dispositivo
try:
    import torch
    device = "cuda" if torch.cuda.is_available() else "cpu"
except Exception:
    device = "cpu"
print("Dispositivo:", device.upper())

embeddings_model = None

# 1) Intento: wrapper oficial de LangChain
try:
    from langchain_huggingface import HuggingFaceEmbeddings
    embeddings_model = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={"device": device},
        encode_kwargs={"normalize_embeddings": True}
    )
    _ = embeddings_model.embed_query("probe")
    print("✅ Embeddings vía langchain_huggingface OK")
except Exception as e:
    print("⚠️ Falla en HuggingFaceEmbeddings:", repr(e))
    print("Activando fallback con SentenceTransformer...")

# 2) Fallback: usar SentenceTransformer directo y adaptarlo
if embeddings_model is None:
    from sentence_transformers import SentenceTransformer
    st_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)

    class STEmbeddingsAdapter:
        def __init__(self, model): self.model = model
        def embed_query(self, text: str):
            return self.model.encode([text], normalize_embeddings=True)[0].tolist()
        def embed_documents(self, texts):
            return self.model.encode(list(texts), normalize_embeddings=True).tolist()

    embeddings_model = STEmbeddingsAdapter(st_model)
    _ = embeddings_model.embed_query("probe")
    print("✅ Embeddings vía SentenceTransformer (fallback) OK")

print("✅ Embeddings listos en", device.upper())

# Tokenización de ejemplo
if 'seg_a_data' in globals() and seg_a_data:
    ejemplo_texto = seg_a_data[0].get("texto", "")
    tk = count_tokens(ejemplo_texto)
    print("\nTokenización de ejemplo:")
    print(f"- Longitud: {len(ejemplo_texto)} chars  |  Tokens aprox: {tk}  |  Límite ~8000")

# Prueba de generación de embedding
print("\nProbando generación de embedding...")
test_text = "Este es un texto de prueba para verificar que los embeddings funcionan correctamente."
emb = embeddings_model.embed_query(test_text)
print("✅ Embedding OK | Dimensión:", len(emb))
print("Primeros 5 valores:", emb[:5])


Configurando modelo de embeddings...
Modelo objetivo: sentence-transformers/all-MiniLM-L6-v2
Dimensión esperada: 384
Dispositivo: CPU
✅ Embeddings vía langchain_huggingface OK
✅ Embeddings listos en CPU

Tokenización de ejemplo:
- Longitud: 3170 chars  |  Tokens aprox: 822  |  Límite ~8000

Probando generación de embedding...
✅ Embedding OK | Dimensión: 384
Primeros 5 valores: [-0.02066517062485218, 0.02713960036635399, -0.03559556230902672, -0.028662530705332756, -0.03590256720781326]


**modulo conflictivo paso 4** error de incompatibilidad de librerias faiss

In [24]:
# ============================================================
# Paso 4 (robusto, sin cambiar versiones): FAISS con fallback
# Import seguro + salidas limpias (LangChain 1.x)
# ============================================================
from typing import List, Dict, Any, Tuple
from dataclasses import dataclass
import numpy as np, os, shutil, json

# LangChain 1.x: Document viene de langchain_core
from langchain_core.documents import Document

# --- Config de verbosidad (dejar en False para salidas limpias) ---
VERBOSE = False
TRY_FAISS = False  # dejar en False en este entorno (NumPy 2.x)

# ---------- 0) Carga segura de FAISS y wrappers LC (condicional) ----------
def _load_faiss(try_faiss: bool):
    if not try_faiss:
        return None
    try:
        import faiss as _faiss
        _ = _faiss.IndexFlatIP(4)  # sanity check
        return _faiss
    except Exception:
        return None

_FAISS = _load_faiss(TRY_FAISS)

# Importar wrappers de langchain_community solo si intentaremos FAISS
LCFAISS = None
InMemoryDocstore = None
if _FAISS is not None:
    try:
        from langchain_community.vectorstores import FAISS as LCFAISS
        from langchain_community.docstore.in_memory import InMemoryDocstore
    except Exception:
        LCFAISS = None
        InMemoryDocstore = None

# ---------- 1) Saneo de documentos ----------
def create_documents_from_segments(segments: List[Dict[str, Any]]) -> List[Document]:
    docs: List[Document] = []
    for seg in segments:
        txt = seg.get("texto", "") or ""
        if not isinstance(txt, str): txt = str(txt)
        txt = txt.strip()
        if not txt: continue
        meta = {
            "id_doc": seg.get("id_doc", ""),
            "chunk_id": seg.get("chunk_id", ""),
            "segmentacion": seg.get("segmentacion", ""),
            "idx": int(seg.get("idx", 0) or 0),
            "autor": seg.get("autor", ""),
            "fecha": seg.get("fecha", ""),
            "tema": seg.get("tema", ""),
            "nombre_archivo": seg.get("nombre_archivo", ""),
        }
        docs.append(Document(page_content=txt, metadata=meta))
    return docs

# ---------- 2) Conversión robusta de embeddings ----------
def to_float32_c_contiguous(vecs) -> np.ndarray:
    import numpy as _np
    try:
        import torch as _torch
    except Exception:
        _torch = None

    if vecs is None:
        raise ValueError("Embedder devolvió None")

    if _torch is not None and isinstance(vecs, _torch.Tensor):
        arr = vecs.detach().cpu().to(_torch.float32).numpy()
        arr = _np.ascontiguousarray(arr, dtype=_np.float32)
        if not _np.isfinite(arr).all(): raise ValueError("NaN/Inf en embeddings")
        return arr

    if isinstance(vecs, (list, tuple)):
        if len(vecs) == 0: raise ValueError("Lista de embeddings vacía")
        rows, d = [], None
        for i, v in enumerate(vecs):
            if v is None: raise ValueError(f"Fila {i} es None")
            if _torch is not None and hasattr(v, "detach") and hasattr(v, "cpu"):
                v = v.detach().cpu().numpy()
            v = _np.asarray(v, dtype=_np.float32).reshape(-1)
            if d is None: d = v.shape[0]
            elif v.shape[0] != d: raise ValueError(f"Dimensiones inconsistentes en fila {i}: {v.shape[0]} vs {d}")
            rows.append(v)
        arr = _np.vstack(rows).astype(_np.float32, copy=False)
        arr = _np.ascontiguousarray(arr, dtype=_np.float32)
        if not _np.isfinite(arr).all(): raise ValueError("NaN/Inf en embeddings")
        return arr

    arr = _np.asarray(vecs, dtype=_np.float32)
    arr = _np.ascontiguousarray(arr, dtype=_np.float32)
    if arr.ndim != 2: raise ValueError(f"Embeddings ndim={arr.ndim}, esperado 2")
    if not _np.isfinite(arr).all(): raise ValueError("NaN/Inf en embeddings")
    return arr

# ---------- 3) Backend de respaldo (NumPy Cosine) ----------
@dataclass
class SimpleDoc:
    text: str
    metadata: Dict[str, Any]

class NumpyCosineVS:
    """VectorStore mínimo compatible con similarity_search_with_score/save/load."""
    def __init__(self, embeddings, docs: List[Document], X: np.ndarray):
        self.embeddings = embeddings
        self.docs = [SimpleDoc(d.page_content, dict(d.metadata)) for d in docs]
        norms = np.linalg.norm(X, axis=1, keepdims=True)
        norms[norms == 0] = 1.0
        self.X = (X / norms).astype(np.float32, copy=False)

    def similarity_search_with_score(self, query: str, k: int = 5):
        qv = self.embeddings.embed_query(query)
        qv = np.asarray(qv, dtype=np.float32).reshape(1, -1)
        qn = qv / max(np.linalg.norm(qv), 1e-12)
        scores = (self.X @ qn.T).reshape(-1)
        k = max(1, min(k, len(scores)))
        idx = np.argpartition(-scores, kth=k-1)[:k]
        idx = idx[np.argsort(-scores[idx])]
        return [(Document(page_content=self.docs[i].text, metadata=self.docs[i].metadata),
                 float(scores[i])) for i in idx]

    def save_local(self, dirpath: str):
        os.makedirs(dirpath, exist_ok=True)
        np.save(os.path.join(dirpath, "vectors.npy"), self.X)
        with open(os.path.join(dirpath, "meta.jsonl"), "w", encoding="utf-8") as f:
            for d in self.docs:
                f.write(json.dumps({"text": d.text, "metadata": d.metadata}, ensure_ascii=False) + "\n")
        with open(os.path.join(dirpath, "_backend.txt"), "w") as f:
            f.write("numpy")

    @classmethod
    def load_local(cls, dirpath: str, embeddings):
        X = np.load(os.path.join(dirpath, "vectors.npy"))
        docs = []
        with open(os.path.join(dirpath, "meta.jsonl"), "r", encoding="utf-8") as f:
            for line in f:
                obj = json.loads(line)
                docs.append(Document(page_content=obj["text"], metadata=obj["metadata"]))
        return cls(embeddings=embeddings, docs=docs, X=X)

# ---------- 4) Intento FAISS con fallback silencioso ----------
def _try_faiss_index(X: np.ndarray, docs: List[Document], embedder, normalize: bool):
    """
    Devuelve (vs, backend) usando FAISS si _FAISS y LCFAISS están disponibles.
    Si no lo están, devuelve (None, None) para activar NumPy.
    """
    if _FAISS is None or LCFAISS is None or InMemoryDocstore is None:
        return None, None
    try:
        index = _FAISS.IndexFlatIP(X.shape[1])
        Xreq = np.require(X, dtype=np.float32, requirements=["C", "A", "W"])
        try:
            index.add(Xreq)
        except Exception:
            Xcopy = np.array(Xreq, dtype=np.float32, copy=True, order="C")
            index.add(Xcopy)

        id_map = {str(i): doc for i, doc in enumerate(docs)}
        docstore = InMemoryDocstore(id_map)
        index_to_docstore_id = {i: str(i) for i in range(X.shape[0])}
        vs = LCFAISS(
            embedding_function=embedder,
            index=index,
            docstore=docstore,
            index_to_docstore_id=index_to_docstore_id,
            normalize_L2=normalize,
        )
        return vs, "faiss"
    except Exception:
        return None, None

def build_vs_with_faiss_or_fallback(docs: List[Document], embedder, normalize=True):
    if not docs:
        raise ValueError("No hay documentos para indexar.")
    texts = [d.page_content for d in docs]
    vecs = embedder.embed_documents(texts)
    X = to_float32_c_contiguous(vecs)
    if normalize:
        norms = np.linalg.norm(X, axis=1, keepdims=True)
        norms[norms == 0] = 1.0
        X = (X / norms).astype(np.float32, copy=False)

    vs, backend = _try_faiss_index(X, docs, embedder, normalize)
    if vs is not None:
        return vs, backend
    return NumpyCosineVS(embeddings=embedder, docs=docs, X=X), "numpy"

# ---------- 5) Creación y guardado de A/B ----------
VECTORSTORE_DIR_A = os.path.join(OUT_DIR, "vectorstore_a")
VECTORSTORE_DIR_B = os.path.join(OUT_DIR, "vectorstore_b")
for p in [VECTORSTORE_DIR_A, VECTORSTORE_DIR_B]:
    if os.path.exists(p): shutil.rmtree(p, ignore_errors=True)

print("Creando bases vectoriales…")
try:
    docs_a = create_documents_from_segments(seg_a_data)
    vectorstore_a, backend_a = build_vs_with_faiss_or_fallback(docs_a, embeddings_model, normalize=True)
    vectorstore_a.save_local(VECTORSTORE_DIR_A)
    print(f"✓ A creada y guardada ({len(docs_a)} docs) | backend: {backend_a}")

    docs_b = create_documents_from_segments(seg_b_data)
    vectorstore_b, backend_b = build_vs_with_faiss_or_fallback(docs_b, embeddings_model, normalize=True)
    vectorstore_b.save_local(VECTORSTORE_DIR_B)
    print(f"✓ B creada y guardada ({len(docs_b)} docs) | backend: {backend_b}")

    print("✓ Ambos vectorstores listos.")
except Exception as e:
    print("✗ Error creando vectorstores:", e)
    raise

# ---------- 6) Prueba de búsqueda (salida breve) ----------
test_query = "inteligencia artificial y aprendizaje automático"
try:
    res_a = vectorstore_a.similarity_search_with_score(test_query, k=3)
    res_b = vectorstore_b.similarity_search_with_score(test_query, k=3)

    print(f"\nConsulta: “{test_query}”")
    print("Segmentación A:")
    for i, (doc, score) in enumerate(res_a, 1):
        autor = doc.metadata.get("autor", "N/A")
        resumen = (doc.page_content[:100].replace("\n"," ") + "…") if len(doc.page_content) > 100 else doc.page_content
        print(f"  {i}. score={score:.4f} | autor={autor} | {resumen}")

    print("\nSegmentación B:")
    for i, (doc, score) in enumerate(res_b, 1):
        autor = doc.metadata.get("autor", "N/A")
        resumen = (doc.page_content[:100].replace("\n"," ") + "…") if len(doc.page_content) > 100 else doc.page_content
        print(f"  {i}. score={score:.4f} | autor={autor} | {resumen}")

    print(f"\nBackends: A={backend_a}, B={backend_b}")
except Exception as e:
    print("✗ Error en búsqueda:", e)


Creando bases vectoriales…
✓ A creada y guardada (227 docs) | backend: numpy
✓ B creada y guardada (349 docs) | backend: numpy
✓ Ambos vectorstores listos.

Consulta: “inteligencia artificial y aprendizaje automático”
Segmentación A:
  1. score=0.6387 | autor=Andrey Ureña Bermúdez | de transparencia y responsabilidad. vii. conclusio'n los temas revisados durante esta semana refuerz…
  2. score=0.6069 | autor=Luis Alfredo González Sánchez | notas de clase inteligenciaartificial-12deagosto-semana2 luis alfredo gonza'lez sa'nchez escuela de …
  3. score=0.5983 | autor=Rodolfo David Acuña López | - sistema mostrando comportamiento "inteligente" 1980s - algo "inteligente" que resuelva tareas comp…

Segmentación B:
  1. score=0.6504 | autor=Andrey Ureña Bermúdez | vii. conclusio'n los temas revisados durante esta semana refuerzan la comprensio'ndeco'molosmodelosd…
  2. score=0.6386 | autor=Rodolfo David Acuña López | references [1] apuntes de la clase de inteligencia artificial, profesor ste

In [25]:
# @title
# ============================================================
# Paso 5: Crear RAG Tool en LangChain
# La herramienta consulta la base vectorial y devuelve:
# - fragmento (texto)
# - documento de origen (id_doc, nombre_archivo)
# - autor
# ============================================================

from typing import Any, List, Tuple, Dict, Optional
import re
import time
from datetime import datetime
import inspect
from langchain_core.tools import Tool  # Para LangChain 0.2; si usas 0.1, cambia a langchain.tools

# ---- Helpers de parsing ------------------------------------------------------

def _parse_query_and_filters(raw_query: str) -> tuple[str, Dict[str, Any]]:
    """
    Extrae filtros de la query y devuelve (texto_libre, filtros_dict).
    Formatos admitidos (espacios opcionales):
      - autor:"Alan Turing"   |  autor: Alan Turing
      - fecha>=2024-01-01     |  fecha<=2025-12-31 (ISO YYYY-MM-DD)
      - tema:"redes neuronales" | tema: redes neuronales
      - tags: audio, cnn
    """
    q = raw_query.strip()

    # Mapa de alias -> campo real de metadata (ajusta según tus metadatos)
    keymap = {
        "autor": "autor",
        "author": "autor",
        "fecha": "fecha",
        "date": "fecha",
        "tema": "tema",
        "topic": "tema",
        "tags": "tags",
        "etiquetas": "tags"
    }

    # Captura patrones key op value (op: :, =, >=, <=, >, <)
    # Ej.: autor:"Alan Turing", fecha>=2024-01-01, tema: redes neuronales
    token_pat = re.compile(
        r'(?P<k>\w+)\s*(?P<op>:|=|>=|<=|>|<)\s*(?P<v>"[^"]+"|\'[^\']+\'|[^,;]+)'
    )

    filters: Dict[str, Any] = {}
    consumed_spans = []

    for m in token_pat.finditer(q):
        k_raw = m.group("k").lower()
        op = m.group("op")
        v_raw = m.group("v").strip()

        # Limpia comillas
        if len(v_raw) >= 2 and ((v_raw[0] == '"' and v_raw[-1] == '"') or (v_raw[0] == "'" and v_raw[-1] == "'")):
            v = v_raw[1:-1].strip()
        else:
            v = v_raw.strip()

        # Normaliza clave
        k = keymap.get(k_raw)
        if not k:
            continue  # clave desconocida -> la ignoramos

        # Guardamos como lista de condiciones por clave (p.ej., múltiples tags)
        if k not in filters:
            filters[k] = []

        # Procesamos valores especiales
        if k == "tags":
            # Separar por comas y limpiar
            tags = [t.strip() for t in re.split(r'[;,]', v) if t.strip()]
            filters[k].append(("in", tags))
        elif k == "fecha":
            # Intentamos parsear fecha ISO
            try:
                _ = datetime.fromisoformat(v)
            except Exception:
                # si no es ISO, lo tratamos como string literal
                pass
            filters[k].append((op, v))
        else:
            # texto exacto/contiene (para op ":" interpretamos "contiene")
            if op == ":":
                filters[k].append(("contains", v))
            elif op == "=":
                filters[k].append(("eq", v))
            else:
                # Para autor/tema normalmente no tiene sentido > <, pero lo admitimos como "eq" por compatibilidad
                filters[k].append(("eq", v))

        consumed_spans.append(m.span())

    # Remueve las partes “consumidas” de la query para dejar el texto libre
    free_text_parts = []
    last_end = 0
    for (start, end) in consumed_spans:
        # añade el texto entre patrones
        free_text_parts.append(q[last_end:start])
        last_end = end
    free_text_parts.append(q[last_end:])
    free_text = " ".join(p.strip() for p in free_text_parts).strip()
    return (free_text, filters)


def _meta_match(meta: Dict[str, Any], filters: Dict[str, Any]) -> bool:
    """
    Aplica filtros a un diccionario de metadata.
    Soporta:
      - autor: ("contains", "Turing") | ("eq","Alan Turing")
      - tema:  idem
      - fecha: (">=", "2024-01-01") etc. (si ISO; si no, compara string)
      - tags:  ("in", ["audio","cnn"]) => al menos uno debe estar en meta['tags']
    """
    def _as_str(x): return "" if x is None else str(x)

    for key, conds in filters.items():
        val = meta.get(key)
        # Normaliza tipo
        if key == "tags":
            # meta puede ser str "a,b,c" o lista
            if isinstance(val, str):
                meta_tags = [t.strip().lower() for t in re.split(r'[;,]', val) if t.strip()]
            elif isinstance(val, list):
                meta_tags = [str(t).strip().lower() for t in val]
            else:
                meta_tags = []
        elif key == "fecha":
            # Mantener string original; si es ISO podemos parsear
            meta_fecha = _as_str(val)
        else:
            meta_text = _as_str(val)

        for (op, expect) in conds:
            if key == "tags" and op == "in":
                wanted = [t.lower() for t in expect]
                if not any(w in meta_tags for w in wanted):
                    return False
            elif key in ("autor", "tema"):
                if op == "contains":
                    if expect.lower() not in meta_text.lower():
                        return False
                elif op == "eq":
                    if meta_text.lower() != expect.lower():
                        return False
                else:
                    # Otros ops no aplican; consideramos fallo
                    return False
            elif key == "fecha":
                # intentamos comparación temporal si ambas son ISO (YYYY-MM-DD)
                lhs, rhs = meta_fecha, str(expect)
                try:
                    d_lhs = datetime.fromisoformat(lhs)
                    d_rhs = datetime.fromisoformat(rhs)
                    if op == ">=" and not (d_lhs >= d_rhs): return False
                    if op == "<=" and not (d_lhs <= d_rhs): return False
                    if op == ">"  and not (d_lhs >  d_rhs): return False
                    if op == "<"  and not (d_lhs <  d_rhs): return False
                    if op in (":", "=", "eq") and not (d_lhs == d_rhs): return False
                except Exception:
                    # Si no son ISO, compara como string
                    if op in (":", "=", "eq") and not (lhs == rhs): return False
                    if op == ">=" and not (lhs >= rhs): return False
                    if op == "<=" and not (lhs <= rhs): return False
                    if op == ">"  and not (lhs >  rhs): return False
                    if op == "<"  and not (lhs <  rhs): return False
            else:
                # clave no soportada
                return False

    return True


def _format_result(i: int, score: Optional[float], seg: str, doc: Any) -> str:
    try:
        score_str = f"{float(score):.4f}" if score is not None else "N/A"
    except Exception:
        score_str = str(score) if score is not None else "N/A"

    meta = getattr(doc, "metadata", {}) or {}
    fragmento = getattr(doc, "page_content", "") or ""
    preview = fragmento[:500] + ("..." if len(fragmento) > 500 else "")

    id_doc         = str(meta.get("id_doc", "N/A"))
    nombre_archivo = str(meta.get("nombre_archivo", "N/A"))
    autor          = str(meta.get("autor", "N/A"))
    chunk_id       = str(meta.get("chunk_id", "N/A"))

    source_line = f"Documento: {id_doc}"
    if nombre_archivo and nombre_archivo != "N/A":
        source_line += f" · archivo: {nombre_archivo}"
    source_line += f" · chunk: {chunk_id}"

    return (
        f"[{seg} · Resultado {i} · score/distancia: {score_str}]\n"
        f"Fragmento:\n{preview}\n"
        f"{source_line}\n"
        f"Autor: {autor}"
    )

# ---- Factory del Tool con filtros -------------------------------------------

def create_rag_tool(
    vectorstore: Any,
    segmentacion_name: str,
    default_top_k: int = 5,
    name_prefix: str = "rag_search"
) -> Tool:
    """
    Tool RAG que soporta filtros por metadata embebidos en la query:
      - autor:"Nombre"
      - fecha>=YYYY-MM-DD
      - tema: algo
      - tags: a, b, c
    """

    def rag_search(user_query: str) -> str:
        query_text, filters = _parse_query_and_filters(user_query or "")

        if not query_text and not filters:
            return "No se proporcionó una consulta."

        # --- Intento 1: usar filtro nativo si el vectorstore lo admite ---
        results: List[Tuple[Any, Optional[float]]] = []
        used_native_filter = False
        k_request = default_top_k

        # ¿similarity_search_with_score acepta 'filter'?
        sig = None
        try:
            sig = inspect.signature(vectorstore.similarity_search_with_score)
        except Exception:
            sig = None

        if sig and "filter" in sig.parameters and filters:
            # Convertimos nuestros filtros a un dict simple de igualdad/contiene
            # Nota: distintos backends aceptan diferentes operadores;
            # aquí probamos un mapeo básico (autor/tema eq|contains, tags in).
            backend_filter = {}

            # Igualdades simples (preferimos eq a contains si hay solo eq)
            for k, conds in filters.items():
                if k == "tags":
                    # algunos backends aceptan {"tags": {"$in": [...]}}
                    # aquí probamos un formato directo (depende del backend)
                    values = []
                    for (op, v) in conds:
                        if op == "in":
                            values.extend(v)
                    if values:
                        backend_filter[k] = values  # puede requerir adaptación
                elif k in ("autor", "tema"):
                    # prioriza "eq" si existe, si no "contains"
                    eq_val = next((v for (op, v) in conds if op == "eq"), None)
                    contains_val = next((v for (op, v) in conds if op == "contains"), None)
                    if eq_val is not None:
                        backend_filter[k] = eq_val
                    elif contains_val is not None:
                        backend_filter[k] = contains_val
                elif k == "fecha":
                    # muchos backends no soportan rangos; lo haremos en fallback
                    pass

            try:
                raw = vectorstore.similarity_search_with_score(
                    query_text if query_text else "", k=max(k_request, 10),
                    filter=backend_filter if backend_filter else None
                )
                # Normaliza a [(doc, score)]
                if raw and isinstance(raw[0], tuple) and len(raw[0]) == 2:
                    results = [(doc, score) for doc, score in raw]
                elif raw:
                    results = [(doc, None) for doc in raw]
                used_native_filter = True
            except Exception:
                used_native_filter = False
                results = []

        # --- Intento 2: fallback universal (k grande + filtrado en Python) ---
        if not results:
            try:
                raw = vectorstore.similarity_search_with_score(
                    query_text if query_text else "", k=max(default_top_k * 10, 50)
                )
                if raw and isinstance(raw[0], tuple) and len(raw[0]) == 2:
                    results = [(doc, score) for doc, score in raw]
                elif raw:
                    results = [(doc, None) for doc in raw]
            except Exception:
                # fallback extra: similarity_search sin score
                try:
                    raw = vectorstore.similarity_search(query_text if query_text else "", k=max(default_top_k * 10, 50))
                    results = [(doc, None) for doc in raw]
                except Exception as e:
                    return f"Error en la búsqueda RAG: {str(e)}"

            # si tenemos filtros, aplicarlos en memoria
            if filters:
                filtered = []
                for (doc, sc) in results:
                    meta = getattr(doc, "metadata", {}) or {}
                    if _meta_match(meta, filters):
                        filtered.append((doc, sc))
                results = filtered

        if not results:
            return "No se encontraron fragmentos relevantes para la consulta."

        # Limitar a top-k finales
        results = results[:default_top_k]

        # Formato de salida
        lines = []
        for i, (doc, score) in enumerate(results, 1):
            lines.append(_format_result(i, score, segmentacion_name, doc))

        # Nota: “score” puede ser distancia: menor = mejor (FAISS).
        # No asumimos que mayor sea mejor universalmente.
        # Si usaste filtro nativo, parte del trabajo ya se hizo “aguas arriba”.
        return "\n\n".join(lines)

    tool_name = f"{name_prefix}_{segmentacion_name}"
    tool_desc = (
        f"Búsqueda RAG en segmentación {segmentacion_name} con filtros por metadata. "
        f"Admite en la query: autor:\"Nombre\", fecha>=YYYY-MM-DD, tema: algo, tags: a,b,c. "
        f"Devuelve fragmentos + fuente (id_doc, archivo, chunk) + autor."
    )

    return Tool(
        name=tool_name,
        description=tool_desc,
        func=rag_search,
        return_direct=False
    )



In [26]:
# @title
# ============================================================
# Comparación RAG A vs B (paper-friendly, sin overlap)
# A = chunks fijos | B = encabezados/secciones
# Filtros: autor="Priscilla" + (opcional) tema
# Produce: métricas concisas, tablas por segmento y CSV separados.
# ============================================================

import time, re, os
from typing import List, Dict, Tuple, Any, Optional
import pandas as pd

# -----------------------------
# 1) Crear tools por segmentación
# -----------------------------
print("Creando herramientas RAG...")
rag_tool_a = create_rag_tool(vectorstore_a, segmentacion_name="A", default_top_k=5)
rag_tool_b = create_rag_tool(vectorstore_b, segmentacion_name="B", default_top_k=5)

print("Herramientas RAG creadas:")
print(f"   - {rag_tool_a.name}: {rag_tool_a.description[:200]}...")
print(f"   - {rag_tool_b.name}: {rag_tool_b.description[:200]}...")

# -----------------------------
# 2) Configuración del experimento
# -----------------------------
AUTOR = "Priscilla"      # filtro obligatorio por metadata
TEMA  = "machine learning"  # <-- pon None si no quieres tema (o cambia el término)

# Construimos la query (autor + tema opcional)
if TEMA and TEMA.strip():
    test_query = f'autor:"{AUTOR}" tema:"{TEMA.strip()}"'
    subtitulo  = f'Filtro: autor="{AUTOR}" + tema="{TEMA.strip()}"'
else:
    test_query = f'autor:"{AUTOR}"'
    subtitulo  = f'Filtro: autor="{AUTOR}"'

# -----------------------------
# 3) Helpers de ejecución y parsing
# -----------------------------
def _safe_invoke(tool, query: str) -> Tuple[str, float]:
    """Invoca el tool (invoke/run) y mide tiempo."""
    t0 = time.perf_counter()
    try:
        out = tool.invoke(query) if hasattr(tool, "invoke") else tool.run(query)
    except Exception as e:
        out = f"[ERROR] {e}"
    dt = time.perf_counter() - t0
    return out, dt

_result_header_pat = re.compile(
    r'^\[(?P<seg>[^|\]]+?)\s*·\s*Resultado\s+(?P<idx>\d+)\s*·\s*score/distancia:\s*(?P<score>[^\]]+)\]',
    re.MULTILINE
)
_doc_line_pat = re.compile(
    r'(?im)^Documento:\s*(?P<id_doc>[^\n·]+)'
    r'(?:\s*·\s*archivo:\s*(?P<archivo>[^\n·]+))?'
    r'\s*·\s*chunk:\s*(?P<chunk_id>[^\n]+)'
)
_author_line_pat = re.compile(r'(?im)^Autor:\s*(?P<autor>.+)$')

def _parse_results(text: str) -> List[Dict[str, Any]]:
    """
    Parsea salida del Tool en ítems estructurados:
    segmento, resultado_idx, score/distancia, id_doc, archivo, chunk_id, autor, fragmento
    """
    if not isinstance(text, str) or not text.strip() or text.strip().startswith("[ERROR]"):
        return []
    items = []
    headers = list(_result_header_pat.finditer(text))
    if not headers:
        return []
    for i, m in enumerate(headers):
        start = m.end()
        end = headers[i + 1].start() if i + 1 < len(headers) else len(text)
        block = text[start:end].strip()

        seg   = m.group("seg").strip()
        idx   = int(m.group("idx"))
        score = m.group("score").strip()

        doc_m  = _doc_line_pat.search(block)
        auth_m = _author_line_pat.search(block)

        id_doc   = doc_m.group("id_doc").strip() if doc_m else "N/A"
        archivo  = doc_m.group("archivo").strip() if (doc_m and doc_m.group("archivo")) else "N/A"
        chunk_id = doc_m.group("chunk_id").strip() if doc_m else "N/A"
        autor    = auth_m.group("autor").strip() if auth_m else "N/A"

        # fragmento sin las líneas de Documento/Autor
        frag = _doc_line_pat.sub("", block)
        frag = _author_line_pat.sub("", frag)
        frag = frag.strip()

        items.append({
            "segmento": seg,
            "resultado_idx": idx,
            "score_ou_distancia": score,
            "id_doc": id_doc,
            "archivo": archivo,
            "chunk_id": chunk_id,
            "autor": autor,
            "chars_fragmento": len(frag),
            "fragmento": frag
        })
    return items

def _metrics(items: List[Dict[str, Any]], elapsed_s: float) -> Dict[str, Any]:
    """Métricas concisas para el paper."""
    chars_total = sum(it.get("chars_fragmento", 0) for it in items)
    docs = {(it["id_doc"] or "N/A").strip() for it in items}
    chunks = {((it["id_doc"] or "N/A").strip(), (it["chunk_id"] or "N/A").strip()) for it in items}
    return {
        "items": len(items),
        "docs_unicos": len(docs),
        "chunks_unicos": len(chunks),
        "chars": chars_total,
        "tiempo_ms": round(elapsed_s * 1000.0, 1)
    }

def _to_df(items: List[Dict[str, Any]]) -> pd.DataFrame:
    cols = ["segmento","resultado_idx","id_doc","chunk_id","archivo","autor","score_ou_distancia","chars_fragmento","fragmento"]
    df = pd.DataFrame(items)
    if df.empty:
        return pd.DataFrame(columns=cols)
    return df[cols].sort_values(["resultado_idx"]).reset_index(drop=True)

def _print_section_title(title: str):
    print("\n" + "="*72)
    print(title)
    print("="*72 + "\n")

# -----------------------------
# 4) Ejecutar A y B (por separado)
# -----------------------------
_print_section_title("PRUEBA EMPÍRICA A vs B (sin overlap)")
print(subtitulo)
print("A: segmentación por chunks fijos | B: por encabezados/secciones\n")

raw_a, t_a = _safe_invoke(rag_tool_a, test_query)
raw_b, t_b = _safe_invoke(rag_tool_b, test_query)

items_a = _parse_results(raw_a)
items_b = _parse_results(raw_b)

metrics_a = _metrics(items_a, t_a)
metrics_b = _metrics(items_b, t_b)

df_a = _to_df(items_a)
df_b = _to_df(items_b)

# -----------------------------
# 5) Exportar CSV por segmento
# -----------------------------
os.makedirs("rag_eval", exist_ok=True)
df_a.to_csv("rag_eval/segmento_A_resultados.csv", index=False)
df_b.to_csv("rag_eval/segmento_B_resultados.csv", index=False)

# -----------------------------
# 6) Reporte por segmento (paper-friendly)
# -----------------------------
# A)
_print_section_title("RESULTADOS — Segmento A (chunks fijos)")
print(f"Query: {test_query}")
print(f"Métricas A: items={metrics_a['items']} | docs_unicos={metrics_a['docs_unicos']} | "
      f"chunks_unicos={metrics_a['chunks_unicos']} | chars={metrics_a['chars']} | "
      f"tiempo={metrics_a['tiempo_ms']} ms")

# Vista tabular resumida (sin mostrar fragmento completo para ser conciso)
cols_summary = ["resultado_idx","id_doc","chunk_id","archivo","autor","score_ou_distancia","chars_fragmento"]
print("\nTabla A (resumen):")
print(df_a[cols_summary].to_string(index=False) if not df_a.empty else "(sin resultados)")

# Vista corta (primer fragmento) — útil para el paper (cita breve)
if not df_a.empty:
    fragA = df_a.loc[0, "fragmento"]
    previewA = fragA[:700] + ("..." if len(fragA) > 700 else "")
    print("\nVista previa A (Top-1 fragmento, truncado):")
    print(previewA)
else:
    print("\nVista previa A: (sin resultados)")

# B)
_print_section_title("RESULTADOS — Segmento B (encabezados/secciones)")
print(f"Query: {test_query}")
print(f"Métricas B: items={metrics_b['items']} | docs_unicos={metrics_b['docs_unicos']} | "
      f"chunks_unicos={metrics_b['chunks_unicos']} | chars={metrics_b['chars']} | "
      f"tiempo={metrics_b['tiempo_ms']} ms")

print("\nTabla B (resumen):")
print(df_b[cols_summary].to_string(index=False) if not df_b.empty else "(sin resultados)")

if not df_b.empty:
    fragB = df_b.loc[0, "fragmento"]
    previewB = fragB[:700] + ("..." if len(fragB) > 700 else "")
    print("\nVista previa B (Top-1 fragmento, truncado):")
    print(previewB)
else:
    print("\nVista previa B: (sin resultados)")

# -----------------------------
# 7) Nota metodológica concisa (lista para el paper)
# -----------------------------
print("\n[Nota metodológica]")
print(
    "Diseño: comparación de recuperación RAG en dos segmentaciones independientes: "
    "A (chunks fijos) y B (encabezados/secciones). La consulta restringe por metadata "
    f"({subtitulo}). Se reportan, por segmento: número de items, documentos únicos, "
    "chunks únicos, caracteres agregados y latencia. Los resultados se presentan por "
    "separado (sin superposiciones), priorizando claridad y concisión."
)

print("\n[CSV generados]")
print(" - rag_eval/segmento_A_resultados.csv")
print(" - rag_eval/segmento_B_resultados.csv")


Creando herramientas RAG...
Herramientas RAG creadas:
   - rag_search_A: Búsqueda RAG en segmentación A con filtros por metadata. Admite en la query: autor:"Nombre", fecha>=YYYY-MM-DD, tema: algo, tags: a,b,c. Devuelve fragmentos + fuente (id_doc, archivo, chunk) + autor....
   - rag_search_B: Búsqueda RAG en segmentación B con filtros por metadata. Admite en la query: autor:"Nombre", fecha>=YYYY-MM-DD, tema: algo, tags: a,b,c. Devuelve fragmentos + fuente (id_doc, archivo, chunk) + autor....

PRUEBA EMPÍRICA A vs B (sin overlap)

Filtro: autor="Priscilla" + tema="machine learning"
A: segmentación por chunks fijos | B: por encabezados/secciones


RESULTADOS — Segmento A (chunks fijos)

Query: autor:"Priscilla" tema:"machine learning"
Métricas A: items=2 | docs_unicos=1 | chunks_unicos=2 | chars=794 | tiempo=21.5 ms

Tabla A (resumen):
 resultado_idx  id_doc      chunk_id archivo                     autor score_ou_distancia  chars_fragmento
             1 DOC_003 DOC_003_A_005     N/A P

In [27]:
# @title
# ============================================================
# Paso 6: Crear WebSearch Tool (DuckDuckGo)
# Solo debe usarse cuando el usuario lo pida explícitamente
# ============================================================

%pip install -q ddgs  # descomenta si aún no lo tienes

from langchain_core.tools import Tool
from ddgs import DDGS
import re, time

def _normalize_query(q: str) -> str:
    q = q.strip()
    m = re.match(r'^(site:[^\s]+)\s+(.+)$', q)
    if m:
        site, rest = m.group(1), m.group(2).strip()
        if not (rest.startswith('"') and rest.endswith('"')) and " " in rest:
            rest = f'"{rest}"'
        return f"{site} {rest}"
    return q

def _ddg_text_search(ddgs_obj: DDGS, q: str, region: str, max_results: int):
    """Compatibilidad entre versiones de ddgs."""
    try:
        # Firmas nuevas (query como primer posicional o keyword)
        return list(ddgs_obj.text(q, region=region, max_results=max_results))
    except TypeError:
        # Firmas antiguas (keywords=...)
        return list(ddgs_obj.text(keywords=q, region=region, max_results=max_results))

def create_web_search_tool() -> Tool:
    def web_search_func(query: str, max_results: int = 6, region: str = "wt-wt") -> str:
        if not isinstance(query, str) or not query.strip():
            return "Consulta inválida para web_search."
        q = _normalize_query(query)
        last_err = None
        for t in range(2):  # reintentos suaves
            try:
                with DDGS() as ddgs_obj:
                    results = _ddg_text_search(ddgs_obj, q, region, max_results)
                if results:
                    lines = []
                    for r in results[:max_results]:
                        title = r.get("title") or "(sin título)"
                        href = r.get("href") or ""
                        snippet = (r.get("body") or "").replace("\n", " ")
                        if len(snippet) > 150:
                            snippet = snippet[:150] + "…"
                        lines.append(f"- {title}\n  {href}\n  {snippet}")
                    return "\n".join(lines)
            except Exception as e:
                last_err = e
            time.sleep(0.6 + 0.4 * t)
        return ("Sin resultados (o bloqueado por el buscador)."
                if last_err is None else f"Error en búsqueda: {type(last_err).__name__}: {last_err}")
    return Tool(
        name="web_search",
        description=("Búsqueda web (DuckDuckGo vía ddgs). Úsala sólo cuando el usuario lo pida "
                     "o si se necesita información externa."),
        func=web_search_func,
    )





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 32.4 MB/s eta 0:00:00


In [28]:
# @title
## Prueba paso 6 Web Search
web_search_tool = create_web_search_tool()
print("WebSearch Tool:", web_search_tool.description)

web = create_web_search_tool()
print(web.invoke("site:wikipedia.org aprendizaje por refuerzo"))



WebSearch Tool: Búsqueda web (DuckDuckGo vía ddgs). Úsala sólo cuando el usuario lo pida o si se necesita información externa.
- Aprendizaje por refuerzo - Wikipedia, la enciclopedia libre
  https://es.wikipedia.org/wiki/Aprendizaje_por_refuerzo
  El aprendizaje por refuerzo o aprendizaje reforzado (en inglés: reinforcement learning) es un área del aprendizaje automático (AA) inspirada en la psi…
- Aprendizaje de refuerzo profundo - Wikipedia, la enciclopedia ...
  https://es.wikipedia.org/wiki/Aprendizaje_de_refuerzo_profundo
  Con esta capa de abstracción, los algoritmos de aprendizaje por refuerzo profundo pueden diseñarse de forma que se generalicen y el mismo modelo pueda…
- Aprendizaje por refuerzo a partir de retroalimentación humana
  https://es.wikipedia.org/wiki/Aprendizaje_por_refuerzo_a_partir_de_retroalimentación_humana
  Visión general del aprendizaje por refuerzo a partir de retroalimentación humana La retroalimentación humana se suele recoger pidiendo a los humanos q…
-

In [29]:
# Instanciar la tool web
web_search_tool = create_web_search_tool()


In [30]:
# @title
# ============================================================
# Paso 7: Resumen y verificación final
# ============================================================

print("="*70)
print("RESUMEN DAVID")
print("="*70)

print("\nDatos procesados:")
print(f" - Fragmentos Segmentación A: {len(seg_a_data)}")
print(f" - Fragmentos Segmentación B: {len(seg_b_data)}")

print("\n Bases vectoriales creadas:")
print(f" - Vectorstore A: {VECTORSTORE_DIR_A}")
print(f" - Vectorstore B: {VECTORSTORE_DIR_B}")

print("\n Herramientas disponibles:")
print(f" 1. {rag_tool_a.name}: Búsqueda RAG con segmentación A (chunks fijos)")
print(f" 2. {rag_tool_b.name}: Búsqueda RAG con segmentación B (encabezados)")
print(f" 3. {web_search_tool.name}: Búsqueda web (solo cuando se solicite explícitamente)")

print("\n📝 Flujo implementado:")
print(" 1. Tokenización: Verificación de tokens con tiktoken")
print(" 2. Embeddings: Generación con sentence-transformers/all-MiniLM-L6-v2 (modelo local, gratuito)")
print(" 3. Almacenamiento: Dos vectorstores FAISS (uno por segmentación)")
print(" 4. Consulta: Herramientas RAG que retornan fragmento, documento y autor")
print(" 5. WebSearch: Herramienta disponible con restricción de uso")


RESUMEN DAVID

Datos procesados:
 - Fragmentos Segmentación A: 227
 - Fragmentos Segmentación B: 349

 Bases vectoriales creadas:
 - Vectorstore A: /content/drive/MyDrive/Colab Notebooks/Tarea3-IA/dataset/vectorstore_a
 - Vectorstore B: /content/drive/MyDrive/Colab Notebooks/Tarea3-IA/dataset/vectorstore_b

 Herramientas disponibles:
 1. rag_search_A: Búsqueda RAG con segmentación A (chunks fijos)
 2. rag_search_B: Búsqueda RAG con segmentación B (encabezados)
 3. web_search: Búsqueda web (solo cuando se solicite explícitamente)

📝 Flujo implementado:
 1. Tokenización: Verificación de tokens con tiktoken
 2. Embeddings: Generación con sentence-transformers/all-MiniLM-L6-v2 (modelo local, gratuito)
 3. Almacenamiento: Dos vectorstores FAISS (uno por segmentación)
 4. Consulta: Herramientas RAG que retornan fragmento, documento y autor
 5. WebSearch: Herramienta disponible con restricción de uso


In [31]:
# ============================================================
# COMPAÑERO 3 – AGENTE, ORQUESTACIÓN, MEMORIA Y APP
# ============================================================
# Paso 1: Instalación de dependencias adicionales para Streamlit
# ============================================================
# NOTA: Las dependencias principales (LangChain, OpenAI, etc.) ya están
# instaladas en la Celda 2. Esta celda solo instala lo necesario para Streamlit.
# ============================================================

# Instalar librerías adicionales para interfaz web
%pip install --quiet streamlit streamlit-chat

print("✅ Streamlit y streamlit-chat instalados correctamente")
print("📦 Listos para la aplicación web")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 30.7 MB/s eta 0:00:00
✅ Streamlit y streamlit-chat instalados correctamente
📦 Listos para la aplicación web


In [32]:
# @title
# ============================================================
# Paso 2: Configuración del modelo OpenAI y definición de prompts
# (dos variantes A/B para pruebas empíricas)
# ============================================================

import os
from langchain_openai import ChatOpenAI
# from langchain.agents import AgentExecutor, create_react_agent
# from langchain.prompts import PromptTemplate
# from langchain.memory import ConversationBufferWindowMemory
from google.colab import userdata

# ============================================================
# CONFIGURACIÓN DE API KEY DE OPENAI
# ============================================================
# IMPORTANTE: Configura OPENAI_API_KEY en Colab Secrets o como variable de entorno
#
# OPCIÓN 1 (Recomendada): Usar Colab Secrets
#   1. Haz clic en el icono 🔒 (candado) en la barra lateral de Colab
#   2. Haz clic en "Add new secret"
#   3. Nombre: OPENAI_API_KEY
#   4. Valor: tu-api-key-de-openai (formato: sk-...)
#
# OPCIÓN 2: Configurar directamente aquí (descomenta y pega tu key):
#   OPENAI_API_KEY = 'sk-tu-api-key-aqui'
# ============================================================

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Si no está en Secrets, puedes configurarla aquí directamente:
# if not OPENAI_API_KEY:
#     OPENAI_API_KEY = 'sk-tu-api-key-aqui'  # <-- Pega tu API key aquí

if not OPENAI_API_KEY:
    print("⚠️  OPENAI_API_KEY no configurada.")
    print("\n📋 INSTRUCCIONES:")
    print("   1. Ve a Colab Secrets (icono 🔒 en la barra lateral)")
    print("   2. Agrega una nueva clave: OPENAI_API_KEY")
    print("   3. O configura la key directamente en el código (ver comentarios arriba)")
    print("\n   Obtén tu API key en: https://platform.openai.com/api-keys")
else:
    # 👉 Exportar al entorno para que los clientes la detecten automáticamente
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    print("✅ OpenAI API Key configurada correctamente")

# ------------------------------------------------------------
# PROMPT VARIANTE A (prioriza rag_search_A; web_search solo si el usuario lo pide explícitamente)
# ------------------------------------------------------------
AGENT_PROMPT_A = """Eres un asistente académico especializado en el curso de Inteligencia Artificial.
Tu nombre es AsistenteIA y tu rol es ayudar a los estudiantes a encontrar información en los apuntes del curso.

**INSTRUCCIONES IMPORTANTES (VARIANTE A):**
1. Antes de responder, consulta SIEMPRE los apuntes usando EXCLUSIVAMENTE la herramienta `rag_search_A`.
2. Cita SIEMPRE el documento de origen y el autor cuando uses información de los apuntes.
3. Solo usa la búsqueda en la web (`web_search`) si el usuario lo solicita explícitamente (por ejemplo: "busca en la web", "revisa en internet", "fuentes externas") **y solo si la herramienta `web_search` está disponible en esta ejecución**. Si no está disponible, indícalo claramente y continúa con `rag_search_A`.

**ESTILO DE RESPUESTA:**
- Sé claro, conciso y educativo.
- Explica conceptos de manera accesible.
- Usa ejemplos cuando sea útil.
- Cita siempre tus fuentes: "Según [Autor] en [Documento]..."

**HERRAMIENTAS DISPONIBLES EN ESTA VARIANTE:**
- rag_search_A: Busca en apuntes usando segmentación por chunks fijos (más preciso para fragmentos específicos).
- (Opcional en tiempo de ejecución) web_search: solo si el usuario lo pide explícitamente y la herramienta está habilitada.

**EJEMPLO DE USO:**
Usuario: "¿Qué es el aprendizaje supervisado?"
1. Usa `rag_search_A` para buscar en los apuntes.
2. Responde basándote en los resultados encontrados.
3. Cita: "Según [Autor] en [Documento]..."

{history}

Pregunta: {input}
Piensa paso a paso y decide qué información recuperar con `rag_search_A`. Solo usa `web_search` si el usuario lo pide explícitamente y la herramienta está disponible.

{agent_scratchpad}"""

# ------------------------------------------------------------
# PROMPT VARIANTE B (prioriza rag_search_B; web_search solo si el usuario lo pide explícitamente)
# ------------------------------------------------------------
AGENT_PROMPT_B = """Eres un asistente académico especializado en el curso de Inteligencia Artificial.
Tu nombre es AsistenteIA y tu rol es ayudar a los estudiantes a encontrar información en los apuntes del curso.

**INSTRUCCIONES IMPORTANTES (VARIANTE B):**
1. Antes de responder, consulta SIEMPRE los apuntes usando EXCLUSIVAMENTE la herramienta `rag_search_B`.
2. Cita SIEMPRE el documento de origen y el autor cuando uses información de los apuntes.
3. Solo usa la búsqueda en la web (`web_search`) si el usuario lo solicita explícitamente (por ejemplo: "busca en la web", "revisa en internet", "fuentes externas") **y solo si la herramienta `web_search` está disponible en esta ejecución**. Si no está disponible, indícalo claramente y continúa con `rag_search_B`.

**ESTILO DE RESPUENSA:**
- Sé claro, conciso y educativo.
- Explica conceptos de manera accesible.
- Usa ejemplos cuando sea útil.
- Cita siempre tus fuentes: "Según [Autor] en [Documento]..."

**HERRAMIENTAS DISPONIBLES EN ESTA VARIANTE:**
- rag_search_B: Busca en apuntes usando segmentación por encabezados (mejor para temas completos).
- (Opcional en tiempo de ejecución) web_search: solo si el usuario lo pide explícitamente y la herramienta está habilitada.

**EJEMPLO DE USO:**
Usuario: "¿Qué es el aprendizaje supervisado?"
1. Usa `rag_search_B` para buscar en los apuntes.
2. Responde basándote en los resultados encontrados.
3. Cita: "Según [Autor] en [Documento]..."

{history}

Pregunta: {input}
Piensa paso a paso y decide qué información recuperar con `rag_search_B`. Solo usa `web_search` si el usuario lo pide explícitamente y la herramienta está disponible.

{agent_scratchpad}"""

print("✅ Prompts definidos para pruebas empíricas A/B")
print("   - AGENT_PROMPT_A → Prioriza SOLO rag_search_A (chunks fijos). web_search: solo si el usuario lo pide y está disponible.")
print("   - AGENT_PROMPT_B → Prioriza SOLO rag_search_B (encabezados). web_search: solo si el usuario lo pide y está disponible.")
print("\n📌 Recuerda: al crear los agentes, pasa únicamente su tool de RAG correspondiente; añade `web_search` SOLO si planeas permitirlo en esa ejecución.")


✅ OpenAI API Key configurada correctamente
✅ Prompts definidos para pruebas empíricas A/B
   - AGENT_PROMPT_A → Prioriza SOLO rag_search_A (chunks fijos). web_search: solo si el usuario lo pide y está disponible.
   - AGENT_PROMPT_B → Prioriza SOLO rag_search_B (encabezados). web_search: solo si el usuario lo pide y está disponible.

📌 Recuerda: al crear los agentes, pasa únicamente su tool de RAG correspondiente; añade `web_search` SOLO si planeas permitirlo en esa ejecución.


In [33]:
# @title
# ============================================================
# - Normaliza nombres de tools
# - DEBUG opcional con impresión de tool_calls
# - Fallback: si no hay tool_calls ni texto, fuerza 1 llamada RAG
# - Fix: ToolMessage siempre con tool_call_id string (LangChain 1.x)
# - Mejora: prioridad web cuando el usuario lo pide
# - Mejora: manejo 429 proactivo + síntesis inmediata (sin 2ª vuelta del LLM)
# ============================================================

import os, json, textwrap
from uuid import uuid4
from google.colab import userdata

from langchain_openai import ChatOpenAI

# --- Mensajes / memoria ---
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_community.chat_message_histories import ChatMessageHistory

DEBUG = False  # pon True para ver trazas de tool_calls

# ============================================================
# 0) API Key
# ============================================================
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY") or userdata.get("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("⚠️  No se puede continuar sin OPENAI_API_KEY.")
    llm = None
else:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# ============================================================
# 3A) Configurar LLM con fallback ordenado
# ============================================================
def configurar_llm(preferidos=None, temperature=0.1):
    if preferidos is None:
        preferidos = [
            "gpt-4o",
            "gpt-4-turbo",
            "gpt-4o-mini",
            "gpt-4",
            "gpt-3.5-turbo",
        ]
    if not os.environ.get("OPENAI_API_KEY"):
        raise RuntimeError("OPENAI_API_KEY no está disponible.")

    print("🔧 Configurando modelo OpenAI (con fallbacks)...")
    last_err = None
    for name in preferidos:
        try:
            _llm = ChatOpenAI(
                model=name,
                openai_api_key=os.environ.get("OPENAI_API_KEY"),
                temperature=temperature,
            )
            # ping mínimo
            _ = _llm.invoke("ping")
            print(f"✅ Modelo configurado: {name}")
            return _llm
        except Exception as e:
            print(f"⚠️  Falló {name}: {e}")
            last_err = e
    raise RuntimeError(f"No se pudo configurar ningún modelo. Último error: {last_err}")

llm = configurar_llm() if OPENAI_API_KEY else None

# ============================================================
# 3B) Agentes A/B con bucle de tool-calling robustecido
# ============================================================

_SESSION_STORES = {}
_SYSTEM_SET = set()

def get_history(session_id: str) -> ChatMessageHistory:
    hist = _SESSION_STORES.get(session_id)
    if hist is None:
        hist = ChatMessageHistory()
        _SESSION_STORES[session_id] = hist
    return hist

def ensure_system_prompt(session_id: str, system_text: str):
    if session_id not in _SYSTEM_SET:
        get_history(session_id).add_message(SystemMessage(content=system_text))
        _SYSTEM_SET.add(session_id)

def _usuario_pide_web(texto: str) -> bool:
    if not isinstance(texto, str):
        return False
    t = texto.lower()
    gatillos = [
        "busca en la web", "buscar en la web", "revisa en la web",
        "en internet", "busca en internet", "buscar en internet",
        "web_search", "fuentes externas", "googlealo", "googlea esto",
        "consulta la web", "haz una búsqueda web",
    ]
    return any(g in t for g in gatillos)

def _norm(s: str) -> str:
    return (s or "").strip().lower()

def _map_tools_por_nombre(tools: list):
    return {_norm(getattr(t, "name", "")): t for t in tools if getattr(t, "name", "")}

# ============================================================
# HOTFIX tool_calls: normalizador y extractor robustos
#  - Soporta:
#    A) {"function": {"name": "...", "arguments": "...|dict"}, "id": "..."}   (OpenAI-like)
#    C) {"name": "...", "arguments": {...}, "id": "..."}                       (variantes planas)
# ============================================================
def _normalize_call_dict(call):
    """Devuelve un dict unificado: {"name": str|None, "arguments": dict|str|None, "id": str|None}"""
    if not isinstance(call, dict):
        return {"name": None, "arguments": None, "id": None}
    cid = call.get("id") or call.get("tool_call_id") or f"auto-{uuid4().hex[:8]}"

    # Caso A (OpenAI-like)
    fn = call.get("function")
    if isinstance(fn, dict) and ("name" in fn or "arguments" in fn):
        nm = fn.get("name")
        args = fn.get("arguments")
        return {"name": nm, "arguments": args, "id": cid}

    if "name" in call and ("args" in call or "arguments" in call):
        nm = call.get("name")
        args = call.get("args", call.get("arguments"))
        return {"name": nm, "arguments": args, "id": cid}

    # Último intento: llaves planas
    nm = call.get("name")
    args = call.get("arguments") or call.get("args")
    return {"name": nm, "arguments": args, "id": cid}

def _extraer_tool_calls(ai_msg: AIMessage):
    calls = getattr(ai_msg, "tool_calls", None)
    kw = getattr(ai_msg, "additional_kwargs", {}) or {}
    if not calls:
        calls = kw.get("tool_calls") or kw.get("tool_calls_json") or []
    if DEBUG:
        print("🧩 tool_calls crudos:", calls if calls else "(ninguno)")
        if kw:
            snippet = textwrap.shorten(str(kw), width=400, placeholder=" …")
            print("🧩 additional_kwargs:", snippet)
    norm = []
    for c in (calls or []):
        norm.append(_normalize_call_dict(c))
    return norm

def _ejecutar_tool_call(tool, call_norm):
    # call_norm viene normalizado por _extraer_tool_calls
    fn = getattr(tool, "invoke", None) or getattr(tool, "run", None) or getattr(tool, "func", None)
    if fn is None:
        return "[Error: herramienta no es invocable]"
    args = call_norm.get("arguments", {})
    # Decodifica si vino como JSON string
    if isinstance(args, str):
        try:
            args = json.loads(args or "{}")
        except Exception:
            pass
    try:
        if isinstance(args, dict):
            if "query" in args:
                try:
                    return fn(args["query"])
                except TypeError:
                    return fn(args)
            return fn(args)
        else:
            return fn(args)
    except Exception as e:
        return f"[Error ejecutando tool: {e}]"

# --- Helpers para ToolMessage con ID válido (requisito en LC 1.x) ---
def _mk_tool_msg(name: str, content: str, tool_call_id: str | None):
    tid = tool_call_id if isinstance(tool_call_id, str) and tool_call_id else f"auto-{_norm(name)}-{uuid4().hex[:8]}"
    nm = name if (name and str(name).strip()) else "tool"
    return ToolMessage(content=str(content), name=nm, tool_call_id=tid)

def _patch_calls_into_history(history, raw_name, tool_out, t_id):
    history.add_message(_mk_tool_msg(raw_name or "tool", tool_out, t_id))

def _patch_forced_rag_into_history(history, principal, tool_out):
    history.add_message(_mk_tool_msg(getattr(principal, "name", "rag"), tool_out, f"forced-{uuid4().hex[:8]}"))

# ====== SÍNTESIS INMEDIATA + 429 PROACTIVO ======
ALWAYS_SYNTHESIZE_AFTER_TOOLS = True
MAX_TOOL_MSGS_FOR_SYNTHESIS = 2
LOG_429_BANNER = True

def _sintetizar_desde_tools(history, max_msgs: int = MAX_TOOL_MSGS_FOR_SYNTHESIS) -> str:
    tmsgs = [m for m in history.messages if isinstance(m, ToolMessage)]
    if not tmsgs:
        return ""
    rec = tmsgs[-max_msgs:]
    partes = []
    for m in rec:
        nombre = getattr(m, "name", "tool")
        contenido = (m.content or "").strip()
        if not contenido:
            continue
        if len(contenido) > 1200:
            contenido = contenido[:1200] + " ..."
        partes.append(f"• {nombre}\n{contenido}")
    if not partes:
        return ""
    return "Síntesis basada en herramientas:\n\n" + "\n\n".join(partes)

class SimpleAgentExecutor:
    def __init__(self, llm_base, system_text: str, tools: list, variante_label: str, max_iters: int = 2):
        # ↓ max_iters=2 para no golpear la cuota
        self.llm = llm_base.bind_tools(tools) if tools else llm_base
        self.system_text = system_text
        self.tools = tools
        self.tools_by_name = _map_tools_por_nombre(tools)  # normalizados
        self.variante_label = variante_label
        self.max_iters = max_iters
        if DEBUG:
            print(f"[DEBUG] Tools registradas ({variante_label}):", list(self.tools_by_name.keys()))

    def invoke(self, payload: dict, config: dict = None):
        cfg = config or {}
        sid = payload.get("session_id") or (cfg.get("configurable", {}) or {}).get("session_id") or "default"

        ensure_system_prompt(sid, self.system_text)
        history = get_history(sid)

        user_input = payload.get("input") or ""
        history.add_message(HumanMessage(content=user_input))
        mensajes = history.messages[:]
        explicito_web = _usuario_pide_web(user_input)

        forced_rag_used = False
        forced_web_used = False

        for it in range(1, self.max_iters + 1):
            try:
                ai_msg = self.llm.invoke(mensajes)
            except Exception as e:
                msg = str(e)

                # === 429 PROACTIVO: si el LLM falla por cuota ANTES de tool_calls ===
                if "429" in msg or "ResourceExhausted" in msg:
                    if LOG_429_BANNER:
                        print("⚠️ [MODO 429 PROACTIVO] LLM falló por cuota; forzando tools y sintetizando…")
                    did_tool = False

                    # 1) Si el usuario pidió web y existe la tool, forzar web primero
                    if explicito_web and "web_search" in self.tools_by_name:
                        web_tool = self.tools_by_name["web_search"]
                        forced_call = {"name": web_tool.name,
                                       "arguments": {"query": user_input},
                                       "id": f"forced-web-{uuid4().hex[:8]}"}
                        tool_out = _ejecutar_tool_call(web_tool, forced_call)
                        _patch_calls_into_history(history, web_tool.name, tool_out, forced_call["id"])
                        did_tool = True

                    # 2) Si no hicimos web, forzar RAG principal de la variante
                    if not did_tool:
                        principal = None
                        for key in ["rag_search_a", "rag_search_b"]:
                            if key in self.tools_by_name:
                                principal = self.tools_by_name[key]; break
                        if principal is not None:
                            forced_call = {"name": principal.name,
                                           "arguments": {"query": user_input},
                                           "id": f"forced-{uuid4().hex[:8]}"}
                            tool_out = _ejecutar_tool_call(principal, forced_call)
                            _patch_forced_rag_into_history(history, principal, tool_out)
                            did_tool = True

                    # 3) Sintetizar con lo que haya
                    sintetico = _sintetizar_desde_tools(history)
                    if sintetico:
                        return {"output": sintetico, "iterations": it}

                    # 4) No hubo tool posible
                    return {
                        "output": ("Estoy temporalmente limitado por cuota (429) y no logré ejecutar "
                                   "una herramienta de respaldo. Intenta de nuevo o cambia de modelo."),
                        "iterations": it
                    }

                # Otros errores
                return {"output": f"[Error del modelo: {e}]", "iterations": it}

            history.add_message(ai_msg)
            calls = _extraer_tool_calls(ai_msg)

            if DEBUG:
                clen = len(ai_msg.content or "")
                print(f"[DEBUG] iter {it} | content len={clen} | tool_calls={bool(calls)}")
                if calls:
                    for c in calls:
                        print("   call(norm):", c.get("name"), c.get("arguments"))

            # === Cuando hay tool_calls, las ejecutamos ===
            if calls:
                # PRIORIDAD WEB: si el usuario pidió web y el LLM NO la incluyó, la forzamos primero
                if explicito_web and "web_search" in self.tools_by_name:
                    has_web = False
                    for c in calls:
                        nm = (c.get("name") or "").strip().lower()
                        if nm == "web_search":
                            has_web = True
                            break
                    if not has_web:
                        web_tool = self.tools_by_name["web_search"]
                        forced_call = {
                            "name": web_tool.name,
                            "arguments": {"query": user_input},
                            "id": f"forced-web-{uuid4().hex[:8]}",
                        }
                        tool_out = _ejecutar_tool_call(web_tool, forced_call)
                        _patch_calls_into_history(history, web_tool.name, tool_out, forced_call["id"])

                # Ejecutar los calls (respetando la política web)
                for call_norm in calls:
                    raw_name = (call_norm.get("name") or "").strip()
                    t_name = _norm(raw_name)
                    t_id = call_norm.get("id", "")

                    if t_name == "web_search" and not explicito_web:
                        tool_out = (
                            "Solicitud de web detectada, pero esta variante solo usa web_search si el usuario "
                            "lo pide explícitamente. Continúo con los apuntes."
                        )
                    else:
                        tool = self.tools_by_name.get(t_name)
                        tool_out = (f"[Tool '{raw_name}' no disponible en esta variante]"
                                    if tool is None else _ejecutar_tool_call(tool, call_norm))

                    _patch_calls_into_history(history, raw_name or "tool", tool_out, t_id)

                # SÍNTESIS INMEDIATA
                if ALWAYS_SYNTHESIZE_AFTER_TOOLS:
                    sintetico = _sintetizar_desde_tools(history)
                    if sintetico:
                        return {"output": sintetico, "iterations": it}

                mensajes = history.messages[:]
                continue  # siguiente iteración tras observar las tools

            # === Si NO hay tool_calls: intentar forzar lo necesario ===
            txt = (ai_msg.content or "").strip()

            # (A) Si el usuario pidió WEB explícita y no hubo tool_calls, forzar web una vez
            if explicito_web and not forced_web_used and "web_search" in self.tools_by_name:
                web_tool = self.tools_by_name["web_search"]
                forced_call = {"name": getattr(web_tool, "name", "web_search"),
                               "arguments": {"query": user_input},
                               "id": f"forced-web-{uuid4().hex[:8]}"}
                tool_out = _ejecutar_tool_call(web_tool, forced_call)
                _patch_calls_into_history(history, web_tool.name, tool_out, forced_call["id"])

                # Síntesis inmediata tras forzar web
                if ALWAYS_SYNTHESIZE_AFTER_TOOLS:
                    sintetico = _sintetizar_desde_tools(history)
                    if sintetico:
                        return {"output": sintetico, "iterations": it}

                mensajes = history.messages[:]
                forced_web_used = True
                continue

            # (B) Si no pidió web, forzar el RAG principal una vez
            if not forced_rag_used:
                principal = None
                for key in ["rag_search_a", "rag_search_b"]:
                    if key in self.tools_by_name:
                        principal = self.tools_by_name[key]; break
                if principal is not None:
                    forced_call = {"name": principal.name,
                                   "arguments": {"query": user_input},
                                   "id": f"forced-{uuid4().hex[:8]}"}
                    tool_out = _ejecutar_tool_call(principal, forced_call)
                    _patch_forced_rag_into_history(history, principal, tool_out)

                    # Síntesis inmediata tras forzar RAG
                    if ALWAYS_SYNTHESIZE_AFTER_TOOLS:
                        sintetico = _sintetizar_desde_tools(history)
                        if sintetico:
                            return {"output": sintetico, "iterations": it}

                    mensajes = history.messages[:]
                    forced_rag_used = True
                    continue

            # (C) Si el modelo sí dio texto, devuélvelo
            if txt:
                return {"output": txt, "iterations": it}

            # (D) Último recurso: síntesis si hay herramientas en historial
            sintetico = _sintetizar_desde_tools(history)
            if sintetico:
                return {"output": sintetico, "iterations": it}

            return {"output": "No fue posible recuperar información en este momento.", "iterations": it}

        # max_iters alcanzado: intenta síntesis antes de rendirte
        sintetico = _sintetizar_desde_tools(history)
        if sintetico:
            return {"output": sintetico, "iterations": self.max_iters}
        return {"output": "Se alcanzó el máximo de iteraciones sin respuesta final.", "iterations": self.max_iters}

def build_agent_executor(llm_base, system_text: str, tools: list, variante_label: str, max_iters: int = 2):
    print(f"\n🤖 Creando agente {variante_label} ...")
    return SimpleAgentExecutor(
        llm_base=llm_base,
        system_text=system_text,
        tools=tools,
        variante_label=variante_label,
        max_iters=max_iters,
    )

# ============================================================
# 3B-bis) Construcción concreta de A y B (usa tus prompts del Paso 2)
# ============================================================

agent_executor_A = None
agent_executor_B = None

if llm is not None:
    tiene_a = "rag_tool_a" in globals()
    tiene_b = "rag_tool_b" in globals()
    tiene_web = "web_search_tool" in globals()
    tiene_promptA = "AGENT_PROMPT_A" in globals()
    tiene_promptB = "AGENT_PROMPT_B" in globals()

    print("\n🔧 Estado de herramientas/prompts detectadas:")
    print(f"   - rag_tool_a: {'sí' if tiene_a else 'no'}")
    print(f"   - rag_tool_b: {'sí' if tiene_b else 'no'}")
    print(f"   - web_search_tool (opcional): {'sí' if tiene_web else 'no'}")
    print(f"   - AGENT_PROMPT_A: {'sí' if tiene_promptA else 'no'}")
    print(f"   - AGENT_PROMPT_B: {'sí' if tiene_promptB else 'no'}")

    if tiene_a and tiene_promptA:
        tools_A = [rag_tool_a] + ([web_search_tool] if tiene_web else [])
        agent_executor_A = build_agent_executor(
            llm_base=llm,
            system_text=AGENT_PROMPT_A,
            tools=tools_A,
            variante_label="A (rag_search_A)",
            max_iters=2,
        )
        print("✅ Agente A listo (rag_search_A + web opcional si el usuario lo pide)")
    else:
        print("⏳ Agente A pendiente (falta rag_tool_a o AGENT_PROMPT_A)")

    if tiene_b and tiene_promptB:
        tools_B = [rag_tool_b] + ([web_search_tool] if tiene_web else [])
        agent_executor_B = build_agent_executor(
            llm_base=llm,
            system_text=AGENT_PROMPT_B,
            tools=tools_B,
            variante_label="B (rag_search_B)",
            max_iters=2,
        )
        print("✅ Agente B listo (rag_search_B + web opcional si el usuario lo pide)")
    else:
        print("⏳ Agente B pendiente (falta rag_tool_b o AGENT_PROMPT_B)")

print("\n📌 Uso:")
print("  respA = agent_executor_A.invoke({'input': 'tu pregunta', 'session_id': 'A'})")
print("  respB = agent_executor_B.invoke({'input': 'tu pregunta', 'session_id': 'B'})")
print("  # Activa DEBUG=True (arriba) para ver tool_calls y args en la consola.")


🔧 Configurando modelo OpenAI (con fallbacks)...
✅ Modelo configurado: gpt-4o

🔧 Estado de herramientas/prompts detectadas:
   - rag_tool_a: sí
   - rag_tool_b: sí
   - web_search_tool (opcional): sí
   - AGENT_PROMPT_A: sí
   - AGENT_PROMPT_B: sí

🤖 Creando agente A (rag_search_A) ...
✅ Agente A listo (rag_search_A + web opcional si el usuario lo pide)

🤖 Creando agente B (rag_search_B) ...
✅ Agente B listo (rag_search_B + web opcional si el usuario lo pide)

📌 Uso:
  respA = agent_executor_A.invoke({'input': 'tu pregunta', 'session_id': 'A'})
  respB = agent_executor_B.invoke({'input': 'tu pregunta', 'session_id': 'B'})
  # Activa DEBUG=True (arriba) para ver tool_calls y args en la consola.


In [34]:
# @title 🔧 Parche: síntesis inmediata + manejo 429 proactivo + menos iteraciones
import os, json
from uuid import uuid4
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# === 1) Sintetizar SIEMPRE tras usar tools (sin esperar otra vuelta del LLM) ===
ALWAYS_SYNTHESIZE_AFTER_TOOLS = True   # evita depender del 2º turno del modelo
MAX_TOOL_MSGS_FOR_SYNTHESIS = 2        # usa las últimas N salidas de herramientas
LOG_429_BANNER = True                  # imprime un banner cuando entra al modo 429 proactivo

def _sintetizar_desde_tools(history, max_msgs: int = MAX_TOOL_MSGS_FOR_SYNTHESIS) -> str:
    from langchain_core.messages import ToolMessage
    tmsgs = [m for m in history.messages if isinstance(m, ToolMessage)]
    if not tmsgs:
        return ""
    rec = tmsgs[-max_msgs:]
    partes = []
    for m in rec:
        nombre = getattr(m, "name", "tool")
        contenido = (m.content or "").strip()
        if not contenido:
            continue
        if len(contenido) > 1200:
            contenido = contenido[:1200] + " ..."
        partes.append(f"• {nombre}\n{contenido}")
    if not partes:
        return ""
    return "Síntesis basada en herramientas:\n\n" + "\n\n".join(partes)

# === 2) Reemplazo del ejecutor: menos iteraciones, síntesis inmediata y 429 proactivo ===
class SimpleAgentExecutor:
    def __init__(self, llm_base, system_text: str, tools: list, variante_label: str, max_iters: int = 2):
        # ↓ max_iters bajo para no quemar cuota
        self.llm = llm_base.bind_tools(tools) if tools else llm_base
        self.system_text = system_text
        self.tools = tools
        self.tools_by_name = _map_tools_por_nombre(tools)
        self.variante_label = variante_label
        self.max_iters = max_iters

    def invoke(self, payload: dict, config: dict = None):
        cfg = config or {}
        sid = payload.get("session_id") or (cfg.get("configurable", {}) or {}).get("session_id") or "default"

        ensure_system_prompt(sid, self.system_text)
        history = get_history(sid)

        user_input = payload.get("input") or ""
        history.add_message(HumanMessage(content=user_input))
        mensajes = history.messages[:]
        explicito_web = _usuario_pide_web(user_input)

        forced_rag_used = False
        forced_web_used = False

        for it in range(1, self.max_iters + 1):
            try:
                ai_msg = self.llm.invoke(mensajes)
            except Exception as e:
                msg = str(e)

                # ——— PARCHE 429 PROACTIVO ———
                if "429" in msg or "ResourceExhausted" in msg:
                    if LOG_429_BANNER:
                        print("⚠️ [MODO 429 PROACTIVO] LLM falló por cuota; forzando tools y sintetizando…")
                    did_tool = False

                    # 1) Si el usuario pidió web y existe la tool, la forzamos
                    if explicito_web and "web_search" in self.tools_by_name:
                        web_tool = self.tools_by_name["web_search"]
                        forced_call = {"name": web_tool.name,
                                       "arguments": {"query": user_input},
                                       "id": f"forced-web-{uuid4().hex[:8]}"}
                        tool_out = _ejecutar_tool_call(web_tool, forced_call)
                        _patch_calls_into_history(history, web_tool.name, tool_out, forced_call["id"])
                        did_tool = True

                    # 2) Si no hicimos web, forzamos el RAG principal de la variante
                    if not did_tool:
                        principal = None
                        for key in ["rag_search_a", "rag_search_b"]:
                            if key in self.tools_by_name:
                                principal = self.tools_by_name[key]; break
                        if principal is not None:
                            forced_call = {"name": principal.name,
                                           "arguments": {"query": user_input},
                                           "id": f"forced-{uuid4().hex[:8]}"}
                            tool_out = _ejecutar_tool_call(principal, forced_call)
                            _patch_forced_rag_into_history(history, principal, tool_out)
                            did_tool = True

                    # 3) Sintetizamos con lo que haya salido de las tools forzadas
                    sintetico = _sintetizar_desde_tools(history)
                    if sintetico:
                        return {"output": sintetico, "iterations": it}

                    # 4) Si por alguna razón no hubo tool posible, mensaje claro
                    return {
                        "output": ("Estoy temporalmente limitado por cuota (429) y no logré ejecutar "
                                   "una herramienta de respaldo. Intenta de nuevo o cambia de modelo."),
                        "iterations": it
                    }
                # ——— FIN PARCHE 429 PROACTIVO ———

                # Otros errores no relacionados con cuota
                return {"output": f"[Error del modelo: {e}]", "iterations": it}

            history.add_message(ai_msg)
            calls = _extraer_tool_calls(ai_msg)

            # === Cuando hay tool_calls, las ejecutamos ===
            if calls:
                for call in calls:
                    raw_name = (call.get("name") or "").strip()
                    t_name = _norm(raw_name)
                    t_id = call.get("id", "")
                    if t_name == "web_search" and not explicito_web:
                        tool_out = ("Solicitud de web detectada, pero esta variante solo usa web_search si el usuario "
                                    "lo pide explícitamente. Continúo con los apuntes.")
                    else:
                        tool = self.tools_by_name.get(t_name)
                        tool_out = (f"[Tool '{raw_name}' no disponible en esta variante]"
                                    if tool is None else _ejecutar_tool_call(tool, call))
                    _patch_calls_into_history(history, raw_name, tool_out, t_id)

                # ⚠️ SÍNTESIS INMEDIATA: evita requerir 2º turno del LLM
                if ALWAYS_SYNTHESIZE_AFTER_TOOLS:
                    sintetico = _sintetizar_desde_tools(history)
                    if sintetico:
                        return {"output": sintetico, "iterations": it}

                # Si quisieras dar oportunidad al LLM, comenta el return de arriba
                mensajes = history.messages[:]
                continue

            # === Sin tool_calls: intentamos forzar lo necesario ===
            txt = (ai_msg.content or "").strip()

            # Forzar web si el usuario lo pidió explícitamente
            if explicito_web and not forced_web_used and "web_search" in self.tools_by_name:
                web_tool = self.tools_by_name["web_search"]
                forced_call = {"name": web_tool.name, "arguments": {"query": user_input}, "id": f"forced-web-{uuid4().hex[:8]}"}
                tool_out = _ejecutar_tool_call(web_tool, forced_call)
                _patch_calls_into_history(history, web_tool.name, tool_out, forced_call["id"])
                # Síntesis inmediata tras forzar web
                sintetico = _sintetizar_desde_tools(history)
                if sintetico:
                    return {"output": sintetico, "iterations": it}
                mensajes = history.messages[:]
                forced_web_used = True
                continue

            # Forzar RAG una vez
            if not forced_rag_used:
                principal = None
                for key in ["rag_search_a", "rag_search_b"]:
                    if key in self.tools_by_name:
                        principal = self.tools_by_name[key]; break
                if principal is not None:
                    forced_call = {"name": principal.name, "arguments": {"query": user_input}, "id": f"forced-{uuid4().hex[:8]}"}
                    tool_out = _ejecutar_tool_call(principal, forced_call)
                    _patch_forced_rag_into_history(history, principal, tool_out)
                    # Síntesis inmediata tras forzar RAG
                    sintetico = _sintetizar_desde_tools(history)
                    if sintetico:
                        return {"output": sintetico, "iterations": it}
                    mensajes = history.messages[:]
                    forced_rag_used = True
                    continue

            # Si el modelo dio texto, devuélvelo
            if txt:
                return {"output": txt, "iterations": it}

            # Último recurso: síntesis si hay herramientas
            sintetico = _sintetizar_desde_tools(history)
            if sintetico:
                return {"output": sintetico, "iterations": it}

            return {"output": "No fue posible recuperar información en este momento.", "iterations": it}

        # max_iters alcanzado: intenta síntesis antes de rendirte
        sintetico = _sintetizar_desde_tools(history)
        if sintetico:
            return {"output": sintetico, "iterations": self.max_iters}
        return {"output": "Se alcanzó el máximo de iteraciones sin respuesta final.", "iterations": self.max_iters}


In [35]:
_SESSION_STORES.clear(); _SYSTEM_SET.clear()
tools_A = [rag_tool_a, web_search_tool]
tools_B = [rag_tool_b, web_search_tool]
agent_executor_A = build_agent_executor(llm, AGENT_PROMPT_A, tools_A, "A (rag_search_a)", max_iters=2)
agent_executor_B = build_agent_executor(llm, AGENT_PROMPT_B, tools_B, "B (rag_search_b)", max_iters=2)
print("A keys:", list(agent_executor_A.tools_by_name.keys()))
print("B keys:", list(agent_executor_B.tools_by_name.keys()))



🤖 Creando agente A (rag_search_a) ...

🤖 Creando agente B (rag_search_b) ...
A keys: ['rag_search_a', 'web_search']
B keys: ['rag_search_b', 'web_search']


In [36]:
# @title 🔬 Tests concisos A/B (mismo prompt) + Web
import uuid
from langchain_core.messages import ToolMessage

def _sid(tag):
    return f"{tag}-{uuid.uuid4().hex[:6]}"

def _tool_names_for(session_id):
    hist = _SESSION_STORES.get(session_id)
    if not hist:
        return []
    return [m.name for m in hist.messages if isinstance(m, ToolMessage)]

def _run(agent, tag, prompt):
    sid = _sid(tag)
    resp = agent.invoke({"input": prompt, "session_id": sid})
    out = (resp.get("output","") or "")[:400].replace("\n", " ")
    tools = _tool_names_for(sid)
    used = ", ".join(tools) if tools else "(ninguna)"
    print(f"\n[{tag}] iters={resp.get('iterations')} · salida:\n  {out}")
    print(f"   tools usadas: {used}")
    return sid, tools

def _ok(msg, cond):
    print(("✅ " if cond else "❌ ") + msg)

assert agent_executor_A and agent_executor_B, "A/B no están construidos."

# ----------------- 1) MISMO PROMPT: RAG A vs RAG B -----------------
base_prompt = "¿Qué es inteligencia artificial? Cita autor y documento de los apuntes."

sidA_rag, toolsA_rag = _run(agent_executor_A, "A·RAG(mismo prompt)", base_prompt)
sidB_rag, toolsB_rag = _run(agent_executor_B, "B·RAG(mismo prompt)", base_prompt)

setA = set(toolsA_rag)
setB = set(toolsB_rag)

# Exclusividad: solo su RAG (permitimos llamadas repetidas, por eso usamos set)
_ok("A usa SOLO rag_search_a (sin web_search ni rag_search_b)",
    setA == {"rag_search_a"})
_ok("B usa SOLO rag_search_b (sin web_search ni rag_search_a)",
    setB == {"rag_search_b"})

# ----------------- 2) MISMO PROMPT + WEB EXPLÍCITO -----------------
web_prompt = base_prompt + " Por favor, busca en la web internet 2 novedades recientes y resume en 2 líneas."

sidA_web, toolsA_web = _run(agent_executor_A, "A·WEB(mismo prompt)", web_prompt)
sidB_web, toolsB_web = _run(agent_executor_B, "B·WEB(mismo prompt)", web_prompt)

setA_web = set(toolsA_web)
setB_web = set(toolsB_web)

# Debe invocar web_search (si además cae a RAG, no es problema)
_ok("A·WEB invoca web_search", "web_search" in setA_web)
_ok("B·WEB invoca web_search", "web_search" in setB_web)

# ----------------- RESUMEN -----------------
print("\n— RESUMEN —")
print(f"A·RAG -> {'OK' if setA == {'rag_search_a'} else 'FAIL'}   | tools: {toolsA_rag}")
print(f"B·RAG -> {'OK' if setB == {'rag_search_b'} else 'FAIL'}   | tools: {toolsB_rag}")
print(f"A·WEB -> {'OK' if 'web_search' in setA_web else 'FAIL'} | tools: {toolsA_web}")
print(f"B·WEB -> {'OK' if 'web_search' in setB_web else 'FAIL'} | tools: {toolsB_web}")



[A·RAG(mismo prompt)] iters=1 · salida:
  Síntesis basada en herramientas:  • rag_search_A [A · Resultado 1 · score/distancia: 0.5599] Fragmento: de transparencia y responsabilidad. vii. conclusio'n los temas revisados durante esta semana refuerzan la comprensio'ndeco'molosmodelosdelenguajemodernosprocesan informacio'n y co'mo se esta'n extendiendo hacia arquitecturas ma's complejas y u'tiles, como los sistemas rag y los agentes inteligen
   tools usadas: rag_search_A

[B·RAG(mismo prompt)] iters=1 · salida:
  Síntesis basada en herramientas:  • rag_search_B [B · Resultado 1 · score/distancia: 0.7283] Fragmento: references [1] apuntes de la clase de inteligencia artificial, profesor steven andrey  pachecoportuguez,institutotecnolo'gicodecostarica,2025. Documento: DOC_001 · chunk: DOC_001_B_006 Autor: Rodolfo David Acuña López  [B · Resultado 2 · score/distancia: 0.6157] Fragmento: vii. conclusio'n los t
   tools usadas: rag_search_B
❌ A usa SOLO rag_search_a (sin web_search ni rag_sea

In [37]:
# Asegúrate de tener la instancia creada en una celda previa:
# web_search_tool = create_web_search_tool()

# 1) Limpiar sesiones/historial para que no arrastren estado viejo
_SESSION_STORES.clear()
_SYSTEM_SET.clear()

# 2) Reconstruir ambos agentes *incluyendo* la tool web
tools_A = [rag_tool_a, web_search_tool]
tools_B = [rag_tool_b, web_search_tool]

agent_executor_A = build_agent_executor(llm, AGENT_PROMPT_A, tools_A, "A (rag_search_a)", max_iters=2)
agent_executor_B = build_agent_executor(llm, AGENT_PROMPT_B, tools_B, "B (rag_search_b)", max_iters=2)

# 3) Verificar que ambos ven 'web_search'
print("A keys:", list(agent_executor_A.tools_by_name.keys()))
print("B keys:", list(agent_executor_B.tools_by_name.keys()))





🤖 Creando agente A (rag_search_a) ...

🤖 Creando agente B (rag_search_b) ...
A keys: ['rag_search_a', 'web_search']
B keys: ['rag_search_b', 'web_search']


In [38]:
# @title
# ============================================================
# Paso 4: Probar el agente con ejemplos
# ============================================================

# Probar agente A
if agent_executor_A:
    print("🧪 Probando el agente A (rag_search_A) con ejemplos...\n")

    # ---------- Ejemplo 1 con Agente A ----------
    print("="*70)
    print("Ejemplo 1 (Agente A): Consulta sobre los apuntes")
    print("="*70)
    test_query_1 = "¿Qué es la inteligencia artificial según los apuntes del curso?"
    print(f"\n❓ Pregunta: {test_query_1}\n")

    try:
        result_1 = agent_executor_A.invoke({"input": test_query_1, "session_id": "test_A"})
        output_1 = result_1.get("output", "")
        print("\n✅ Respuesta del agente A:")
        print(output_1[:500] + "..." if len(output_1) > 500 else output_1)
        print(f"\n🔧 Iteraciones: {result_1.get('iterations', 'N/A')}")
    except Exception as e:
        print(f"❌ Error: {e}")

    # ---------- Ejemplo 2 con Agente A ----------
    print("\n" + "="*70)
    print("Ejemplo 2 (Agente A): Consulta específica")
    print("="*70)
    test_query_2 = "Explícame sobre aprendizaje supervisado"
    print(f"\n❓ Pregunta: {test_query_2}\n")

    try:
        result_2 = agent_executor_A.invoke({"input": test_query_2, "session_id": "test_A"})
        output_2 = result_2.get("output", "")
        print("\n✅ Respuesta del agente A:")
        print(output_2[:500] + "..." if len(output_2) > 500 else output_2)
        print(f"\n🔧 Iteraciones: {result_2.get('iterations', 'N/A')}")
    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("⚠️  El agente A no está configurado. Configura OPENAI_API_KEY primero.")

# Probar agente B
if agent_executor_B:
    print("\n" + "="*70)
    print("🧪 Probando el agente B (rag_search_B) con ejemplos...\n")

    # ---------- Ejemplo 1 con Agente B ----------
    print("="*70)
    print("Ejemplo 1 (Agente B): Consulta sobre los apuntes")
    print("="*70)
    test_query_1 = "¿Qué es la inteligencia artificial según los apuntes del curso?"
    print(f"\n❓ Pregunta: {test_query_1}\n")

    try:
        result_1 = agent_executor_B.invoke({"input": test_query_1, "session_id": "test_B"})
        output_1 = result_1.get("output", "")
        print("\n✅ Respuesta del agente B:")
        print(output_1[:500] + "..." if len(output_1) > 500 else output_1)
        print(f"\n🔧 Iteraciones: {result_1.get('iterations', 'N/A')}")
    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("⚠️  El agente B no está configurado. Configura OPENAI_API_KEY primero.")


🧪 Probando el agente A (rag_search_A) con ejemplos...

Ejemplo 1 (Agente A): Consulta sobre los apuntes

❓ Pregunta: ¿Qué es la inteligencia artificial según los apuntes del curso?


✅ Respuesta del agente A:
Síntesis basada en herramientas:

• rag_search_A
[A · Resultado 1 · score/distancia: 0.6600]
Fragmento:
de transparencia y responsabilidad. vii. conclusio'n los temas revisados durante esta semana refuerzan la comprensio'ndeco'molosmodelosdelenguajemodernosprocesan informacio'n y co'mo se esta'n extendiendo hacia arquitecturas ma's complejas y u'tiles, como los sistemas rag y los agentes inteligentes. estas herramientas representan un paso clave hacia una inteligencia artificial ma's contextual,...

🔧 Iteraciones: 1

Ejemplo 2 (Agente A): Consulta específica

❓ Pregunta: Explícame sobre aprendizaje supervisado


✅ Respuesta del agente A:
Síntesis basada en herramientas:

• rag_search_A
[A · Resultado 1 · score/distancia: 0.6600]
Fragmento:
de transparencia y responsabilidad. vii. 

In [ ]:
# @title
# ============================================================
# Paso 5: App Streamlit con “Web solo si el usuario lo pide”
# ============================================================

# NOTA: Esta celda usa código antiguo de LangChain que NO es compatible
# con la versión actual instalada (0.3.27). Se comenta para evitar errores.
# Si necesitas Streamlit, deberás adaptar el código al SimpleAgentExecutor
# que ya está implementado en las celdas anteriores.

# CÓDIGO COMENTADO - INCOMPATIBLE CON LANGCHAIN 0.3.27
"""
import os, pickle, re
from langchain_openai import ChatOpenAI
# from langchain.agents import AgentExecutor, create_react_agent  # NO DISPONIBLE EN 0.3.27
"""
# ✅ Instalación de dependencias necesarias para LangChain 0.3.27
%pip install --quiet langchain langchain-core langchain-community langchain-openai

# ✅ Imports actualizados para LangChain 0.3.27
from langchain_core.prompts import PromptTemplate

# ConversationBufferWindowMemory - intentar múltiples ubicaciones
ConversationBufferWindowMemory = None
try:
    from langchain.memory import ConversationBufferWindowMemory
    print("✅ ConversationBufferWindowMemory importado desde langchain.memory")
except ImportError:
    try:
        from langchain_core.memory import ConversationBufferWindowMemory
        print("✅ ConversationBufferWindowMemory importado desde langchain_core.memory")
    except ImportError:
        try:
            # Intentar importar desde langchain.chains si está disponible
            from langchain.chains.conversation.memory import ConversationBufferWindowMemory
            print("✅ ConversationBufferWindowMemory importado desde langchain.chains.conversation.memory")
        except ImportError:
            print("⚠️ ConversationBufferWindowMemory no encontrado en ninguna ubicación estándar")
            print("   Verificando módulos disponibles...")
            try:
                import langchain.memory as mem
                print(f"   Módulos en langchain.memory: {dir(mem)}")
            except:
                pass
            # Si no está disponible, el código fallará más adelante cuando se intente usar
            # El usuario necesitará adaptar el código o usar una alternativa
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.tools.render import render_text_description
from langchain_core.tools import Tool

# --------- guardar config para la app ----------
# Verificar que las variables necesarias estén definidas
required_vars = {
    "OUT_DIR": OUT_DIR if 'OUT_DIR' in globals() else None,
    "VECTORSTORE_DIR_A": VECTORSTORE_DIR_A if 'VECTORSTORE_DIR_A' in globals() else None,
    "VECTORSTORE_DIR_B": VECTORSTORE_DIR_B if 'VECTORSTORE_DIR_B' in globals() else None,
    "PROYECTO_DIR": PROYECTO_DIR if 'PROYECTO_DIR' in globals() else None,
}

# Determinar qué prompt usar (AGENT_PROMPT, AGENT_PROMPT_A, o crear uno genérico)
if 'AGENT_PROMPT' in globals():
    agent_prompt = AGENT_PROMPT
elif 'AGENT_PROMPT_A' in globals():
    agent_prompt = AGENT_PROMPT_A
    print("ℹ️ Usando AGENT_PROMPT_A como prompt del agente")
elif 'AGENT_PROMPT_B' in globals():
    agent_prompt = AGENT_PROMPT_B
    print("ℹ️ Usando AGENT_PROMPT_B como prompt del agente")
else:
    agent_prompt = "Eres un asistente académico especializado en el curso de Inteligencia Artificial."
    print("⚠️ No se encontró AGENT_PROMPT, AGENT_PROMPT_A ni AGENT_PROMPT_B. Usando prompt genérico.")

# Verificar que las variables requeridas estén disponibles
missing_vars = [var for var, value in required_vars.items() if value is None]
if missing_vars:
    print(f"⚠️ Variables faltantes: {', '.join(missing_vars)}")
    print("   Asegúrate de ejecutar las celdas anteriores que definen estas variables.")
    print("   El código de Streamlit no se generará hasta que todas las variables estén definidas.")
else:
    STREAMLIT_DATA_PATH = os.path.join(required_vars["OUT_DIR"], "streamlit_data.pkl")
    with open(STREAMLIT_DATA_PATH, "wb") as f:
        pickle.dump({
            "agent_prompt": agent_prompt,
            "vectorstore_a_path": required_vars["VECTORSTORE_DIR_A"],
            "vectorstore_b_path": required_vars["VECTORSTORE_DIR_B"],
            "proyecto_dir": required_vars["PROYECTO_DIR"],
        }, f)
    print("✅ Configuración guardada para Streamlit:", STREAMLIT_DATA_PATH)

# --------- archivo de la app ----------
STREAMLIT_APP_CODE = f'''import streamlit as st
import os, re, pickle
from typing import List

from langchain_openai import ChatOpenAI
# ⚠️ NOTA: AgentExecutor y create_react_agent no están disponibles en LangChain 0.3.27
# Necesitarás usar una alternativa como SimpleAgentExecutor o actualizar a una versión compatible
# from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
# ConversationBufferWindowMemory - intentar múltiples ubicaciones
ConversationBufferWindowMemory = None
try:
    from langchain.memory import ConversationBufferWindowMemory
    print("✅ ConversationBufferWindowMemory importado desde langchain.memory")
except ImportError:
    try:
        from langchain_core.memory import ConversationBufferWindowMemory
        print("✅ ConversationBufferWindowMemory importado desde langchain_core.memory")
    except ImportError:
        try:
            from langchain.chains.conversation.memory import ConversationBufferWindowMemory
            print("✅ ConversationBufferWindowMemory importado desde langchain.chains.conversation.memory")
        except ImportError:
            print("⚠️ ConversationBufferWindowMemory no encontrado en ninguna ubicación estándar")
            print("   El código de Streamlit puede no funcionar correctamente sin esta clase.")
            print("   Considera usar una alternativa o actualizar LangChain a una versión compatible.")
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.tools import Tool
from langchain_core.tools.render import render_text_description

# DuckDuckGo (opcional)
try:
    from langchain_community.tools import DuckDuckGoSearchRun
    _HAS_DDG = True
except Exception:
    _HAS_DDG = False

st.set_page_config(page_title="AsistenteIA - Curso de IA", page_icon="🤖", layout="wide")
st.title("🤖 AsistenteIA - Curso de Inteligencia Artificial")
st.caption("Prioriza apuntes (RAG). Usa la web **solo** si lo pides explícitamente.")

BASE_DIR = "{PROYECTO_DIR}"
OUT_DIR = os.path.join(BASE_DIR, "dataset")
CFG_PATH = os.path.join(OUT_DIR, "streamlit_data.pkl")

@st.cache_resource
def load_cfg():
    with open(CFG_PATH, "rb") as f:
        return pickle.load(f)

@st.cache_resource
def load_embeddings():
    return HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={{"device": "cpu"}},
        encode_kwargs={{"normalize_embeddings": True}}
    )

@st.cache_resource
def load_vectorstores():
    cfg = load_cfg()
    emb = load_embeddings()
    vs_a = FAISS.load_local(cfg["vectorstore_a_path"], emb, allow_dangerous_deserialization=True)
    vs_b = FAISS.load_local(cfg["vectorstore_b_path"], emb, allow_dangerous_deserialization=True)
    return vs_a, vs_b

def create_rag_tool(vs, name: str) -> Tool:
    def rag_search(query: str, k: int = 5) -> str:
        try:
            results = vs.similarity_search_with_score(query, k=k)
            if not results:
                return "No se encontraron fragmentos relevantes."
            out = []
            for i, (doc, score) in enumerate(results, 1):
                autor = doc.metadata.get("autor", "N/A")
                id_doc = doc.metadata.get("id_doc", "N/A")
                chunk_id = doc.metadata.get("chunk_id", "N/A")
                frag = (doc.page_content or "")[:700].replace("\\n"," ")
                out.append(
                    f"[Resultado {{i}} · Score: {{score:.4f}}]\\n"
                    f"Fragmento: {{frag}}...\\n"
                    f"Según [{{autor}}] en [{{id_doc}}] (chunk {{chunk_id}})."
                )
            return "\\n\\n".join(out)
        except Exception as e:
            return f"Error en RAG {{name}}: {{e}}"
    return Tool(
        name=f"rag_search_{{name}}",
        description=f"Busca en apuntes (segmentación {{name}}). Devuelve fragmentos con cita.",
        func=rag_search
    )

def create_web_tool() -> Tool:
    if not _HAS_DDG:
        return Tool(
            name="web_search",
            description="Búsqueda web (stub). Úsala solo si la pides explícitamente.",
            func=lambda q: "WebSearch no disponible (falta DuckDuckGoSearchRun)."
        )
    search = DuckDuckGoSearchRun()
    def _web(q: str) -> str:
        try:
            r = search.run(q)
            return f"Resultados web para '{{q}}'\\n\\n{{r}}"
        except Exception as e:
            return f"Error en búsqueda web: {{e}}"
    return Tool(
        name="web_search",
        description="Búsqueda en internet (DuckDuckGo). Solo cuando lo pidas explícitamente.",
        func=_web
    )

# --------- Estado ----------
if "messages" not in st.session_state: st.session_state.messages = []
if "agent_rag" not in st.session_state: st.session_state.agent_rag = None
if "agent_rag_web" not in st.session_state: st.session_state.agent_rag_web = None

# --------- Sidebar ----------
with st.sidebar:
    st.header("⚙️ Configuración")
    gkey = st.text_input("OpenAI API Key", type="password", value=os.getenv("OPENAI_API_KEY",""))
    if gkey: os.environ["OPENAI_API_KEY"] = gkey

    if gkey and (st.session_state.agent_rag is None or st.session_state.agent_rag_web is None):
        with st.spinner("Inicializando agentes..."):
            cfg = load_cfg()
            vs_a, vs_b = load_vectorstores()

            ragA = create_rag_tool(vs_a, "A")
            ragB = create_rag_tool(vs_b, "B")
            web  = create_web_tool()

            def make_agent(tools):
                llm = ChatOpenAI(
                    model="gpt-4-turbo",
                    openai_api_key=gkey,
                    temperature=0.1
                )
                tool_str = render_text_description(tools)
                tool_names = ", ".join([t.name for t in tools])

                # ✅ Incluimos {{history}} y {{agent_scratchpad}} en el template
                prompt = PromptTemplate(
                    template=(
                        "{{agent_profile}}\\n\\n"
                        "Tienes acceso a estas herramientas:\\n{{tools}}\\n\\n"
                        "Sigue este formato EXACTO (sin bloques de código):\\n\\n"
                        "Question: {{input}}\\n"
                        "Thought: razona brevemente el siguiente paso\\n"
                        "Action: una de [{{tool_names}}]\\n"
                        "Action Input: el input para la acción\\n"
                        "Observation: el resultado de la acción\\n"
                        "... (repite Thought/Action/Action Input/Observation si hace falta) ...\\n"
                        "Thought: I now know the final answer\\n"
                        "Final Answer: tu respuesta final en español, citando autor y documento si aplica\\n\\n"
                        "Historial:\\n{{history}}\\n\\n"
                        "Razonamiento y pasos previos:\\n{{agent_scratchpad}}\\n"
                    ),
                    input_variables=["history","input","agent_scratchpad","tools","tool_names"]
                ).partial(
                    agent_profile=cfg["agent_prompt"],
                    tools=tool_str,
                    tool_names=tool_names
                )

                memory = ConversationBufferWindowMemory(
                    k=5,
                    memory_key="history",
                    return_messages=True,
                    output_key="output"
                )

                agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
                return AgentExecutor(
                    agent=agent,
                    tools=tools,
                    memory=memory,
                    verbose=True,
                    handle_parsing_errors=True,
                    max_iterations=8,
                    return_intermediate_steps=True
                )

            # agente SOLO RAG
            st.session_state.agent_rag = make_agent([ragA, ragB])
            # agente RAG + Web
            st.session_state.agent_rag_web = make_agent([ragA, ragB, web])

            st.success("✅ Agentes listos: RAG y RAG+Web")

# --------- Render previo ----------
for m in st.session_state.messages:
    with st.chat_message(m["role"]):
        st.markdown(m["content"])

# --------- Chat ----------
if user_prompt := st.chat_input("Pregunta algo sobre los apuntes de IA..."):
    st.session_state.messages.append({{"role":"user","content":user_prompt}})
    with st.chat_message("user"): st.markdown(user_prompt)

    # ¿El usuario pidió explícitamente web?
    wants_web = re.search(r"\\b(web|internet|google|websearch|buscar en (la )?web)\\b", user_prompt, re.I)

    executor = st.session_state.agent_rag_web if wants_web else st.session_state.agent_rag

    with st.chat_message("assistant"):
        if executor is None:
            st.warning("Configura la API key en el sidebar.")
        else:
            with st.spinner("Pensando..."):
                try:
                    result = executor.invoke({{"input": user_prompt}})
                    resp = result.get("output","")
                    st.markdown(resp)
                    st.session_state.messages.append({{"role":"assistant","content":resp}})
                except Exception as e:
                    st.error(f"Error: {{e}}")
'''

STREAMLIT_APP_PATH = os.path.join(PROYECTO_DIR, "streamlit_app.py")
with open(STREAMLIT_APP_PATH, "w", encoding="utf-8") as f:
    f.write(STREAMLIT_APP_CODE)

print("✅ App Streamlit escrita:")
print("   ", STREAMLIT_APP_PATH)
print("🚀 Ejecuta:  streamlit run streamlit_app.py  (o tu bloque con ngrok)")


ModuleNotFoundError: No module named 'langchain.prompts'

In [ ]:
# @title
# ============================================================
# Paso 6: Resumen final y verificación de Compañero 3 (actualizado)
# ============================================================

import os

# Intentar recuperar rutas ya definidas; si no existen, usar valores por defecto seguros
try:
    app_path = STREAMLIT_APP_PATH
except NameError:
    try:
        app_path = os.path.join(PROYECTO_DIR, "streamlit_app.py")
    except NameError:
        app_path = "streamlit_app.py"

try:
    proyecto_dir = PROYECTO_DIR
except NameError:
    proyecto_dir = "."

try:
    streamlit_data_path = STREAMLIT_DATA_PATH
except NameError:
    try:
        streamlit_data_path = os.path.join(os.path.join(proyecto_dir, "dataset"), "streamlit_data.pkl")
    except Exception:
        streamlit_data_path = "dataset/streamlit_data.pkl"

try:
    vectorstore_a_path = VECTORSTORE_DIR_A
except NameError:
    vectorstore_a_path = "<VECTORSTORE_DIR_A>"

try:
    vectorstore_b_path = VECTORSTORE_DIR_B
except NameError:
    vectorstore_b_path = "<VECTORSTORE_DIR_B>"

print("="*70)
print("✅ RESUMEN DE LA PARTE DEL COMPAÑERO 3")
print("="*70)

print("\n📋 Componentes implementados:")

print("\n1. ✅ Prompt base/perfil del agente:")
print("   - Nombre: AsistenteIA")
print("   - Rol: Asistente académico especializado en IA")
print("   - Estilo: Claro, educativo, con citas")
print("   - Restricción: Primero apuntes (RAG), luego web (solo si el usuario lo solicita o no hay cobertura)")

print("\n2. ✅ Agente orquestador:")
print("   - Modelos preferidos (fallback en cascada):")
print("       gpt-4-turbo → gpt-4o-mini → gpt-4 → gpt-3.5-turbo → gpt-4o → gpt-3.5-turbo")
print("   - Parámetros: temperature=0.1, max_output_tokens=1024")
print("   - Patrón: ReAct ESTRICTO (Reasoning + Acting) con formato impuesto")
print("   - Post-procesado: verificación de cita con ensure_citation")
print("   - Decisión: Entre rag_search_A, rag_search_B, web_search (stub) o cierre directo")
print("   - Max iteraciones: 8")
print("   - early_stopping_method: generate")
print("   - return_intermediate_steps: True")

print("\n3. ✅ Memoria conversacional:")
print("   - Tipo: ConversationBufferWindowMemory")
print("   - Ventana: Últimas 5 interacciones (k=5)")
print("   - Característica: No guarda historial permanente")

print("\n4. ✅ Interfaz Streamlit:")
print("   - Aplicación web completa con chat en tiempo real")
print("   - Indica herramientas usadas por turno")
print("   - Sidebar para configurar OPENAI_API_KEY y ver últimas herramientas")
print(f"   - Archivo de aplicación: {app_path}")
print(f"   - Configuración guardada: {streamlit_data_path}")

print("\n5. ✅ Integración completa:")
print("   - Agente + Tools + Memoria + Interfaz integrados")
print("   - Listo para demostración en vivo con tus vectorstores")

print("\n📊 Herramientas disponibles para el agente:")
print("   1) rag_search_A: Búsqueda en apuntes (segmentación A - chunks fijos) con cita obligatoria")
print("   2) rag_search_B: Búsqueda en apuntes (segmentación B - encabezados) con cita obligatoria")
print("   3) web_search: Búsqueda web (stub en esta demo; usar solo si se solicita)")

print("\n📁 Rutas de vectorstores (FAISS):")
print(f"   - VECTORSTORE_DIR_A: {vectorstore_a_path}")
print(f"   - VECTORSTORE_DIR_B: {vectorstore_b_path}")

print("\n🚀 Para ejecutar la aplicación:")
print("   1) Asegura que los vectorstores (A y B) existan en las rutas anteriores.")
print("   2) Configura la variable de entorno OPENAI_API_KEY (o en el sidebar de la app).")
print(f"   3) cd {proyecto_dir}")
print(f"   4) streamlit run {os.path.basename(app_path)}")

print("\n" + "="*70)
print("✅ COMPAÑERO 3 - TAREA COMPLETADA")
print("="*70)
print("\n🎯 El sistema está listo para:")
print("   - Comparar el comportamiento de ambas segmentaciones (A vs B)")
print("   - Observar herramientas usadas, trazas ReAct e impacto de la memoria")
print("   - Demostración presencial con consultas reales del curso")
print("="*70)
!pip install pyngrok


In [ ]:
# Instalar duckduckgo-search para búsqueda web
# NOTA: langchain_community ya está instalado en la Celda 2 (versión >=0.2.15)
# Solo necesitamos instalar duckduckgo-search
%pip install --quiet duckduckgo-search

In [ ]:
# @title
!pip -q install pyngrok streamlit requests

import os, time, shlex, subprocess, requests
from pyngrok import ngrok

APP_PATH = "/content/drive/MyDrive/Colab Notebooks/Tarea3-IA/streamlit_app.py"
PORT = 8502

# 1) Arranca Streamlit
!pkill -f "streamlit run" || true
cmd = f'streamlit run "{APP_PATH}" --server.port {PORT} --server.address 0.0.0.0 --server.headless true --browser.gatherUsageStats false'
sp = subprocess.Popen(shlex.split(cmd))

# 2) Espera a que esté saludable
import time
ok = False
for _ in range(40):
    time.sleep(0.5)
    try:
        r = requests.get(f"http://localhost:{PORT}/_stcore/health", timeout=1)
        if r.ok:
            ok = True
            break
    except Exception:
        pass
if not ok:
    raise SystemExit("Streamlit no respondió; revisa logs con: !pkill -f 'streamlit run'; !tail -n 120 /tmp/streamlit.log")

# 3) Configura authtoken (desde Secrets o pegado)
try:
    from google.colab import userdata
    token = userdata.get("NGROK_AUTHTOKEN")
except Exception:
    token = None
# token = "PEGAR_AQUI_TU_TOKEN"  # <-- alternativa manual
if not token:
    raise SystemExit("Falta NGROK_AUTHTOKEN (ponlo en Colab Secrets o pégalo en la variable 'token').")

ngrok.set_auth_token(token)

# 4) Cierra túneles previos y abre uno nuevo
for t in ngrok.get_tunnels():
    ngrok.disconnect(t.public_url)

public_url = ngrok.connect(PORT, "http").public_url
print("🌐 URL pública:", public_url)
